In [2]:
%matplotlib inline

import librosa
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
import torch
import os
import glob
import pickle
import copy
import random
import time
import traceback
from multiprocessing import Process

# Make file index dict

In [7]:
train_list = glob.glob('/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/*')

In [8]:
train_list

['/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_106344.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_151403.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_162270.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_150448.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_157718.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_104892.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_144989.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_147787.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_130559.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101127.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_176958.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_108219.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_143663.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/tas

In [2]:
OPT_INDEX = '/Lun0/zhiyong/sdsvc_t2_train'

data_dir = '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train'
train_label_dir = '/Lun0/zhiyong/SdSV_2020_deepmine/task2_enrollment/docs/train_labels.txt'
new_start_id = 7323 #vox with 0-7322


spk2id = os.path.join(OPT_INDEX, 'spk2id')
spk2utt_dict = os.path.join(OPT_INDEX, 'spk2utt_dict')
spk2utt_train_dict = os.path.join(OPT_INDEX, 'spk2utt_train_dict')

In [17]:
train_labels2utt = {}
with open(train_label_dir, 'r') as f:
    for count, line in enumerate(f):
        if count == 0:
            continue
        line = line[:-1]
        utt, label = line.split(' ')
        if label not in train_labels2utt.keys():
            train_labels2utt[label] = [data_dir+'/'+utt+'.wav']
        else:
            train_labels2utt[label].append(data_dir+'/'+utt+'.wav')

In [18]:
len(train_labels2utt)

588

In [21]:
count = 0
for i in train_labels2utt:
    count += len(train_labels2utt[i])
print(count)

85764


In [22]:
spk2utt = {}

with open(spk2id, 'w') as f:
    pass

for count, i in enumerate(train_labels2utt):
    with open(spk2id, 'a') as f:
        f.write(str(count+new_start_id)+','+i+'\n')
    spk2utt[count+new_start_id] = train_labels2utt[i]

In [23]:
len(spk2utt)

588

In [26]:
spk2utt.keys()

dict_keys([7323, 7324, 7325, 7326, 7327, 7328, 7329, 7330, 7331, 7332, 7333, 7334, 7335, 7336, 7337, 7338, 7339, 7340, 7341, 7342, 7343, 7344, 7345, 7346, 7347, 7348, 7349, 7350, 7351, 7352, 7353, 7354, 7355, 7356, 7357, 7358, 7359, 7360, 7361, 7362, 7363, 7364, 7365, 7366, 7367, 7368, 7369, 7370, 7371, 7372, 7373, 7374, 7375, 7376, 7377, 7378, 7379, 7380, 7381, 7382, 7383, 7384, 7385, 7386, 7387, 7388, 7389, 7390, 7391, 7392, 7393, 7394, 7395, 7396, 7397, 7398, 7399, 7400, 7401, 7402, 7403, 7404, 7405, 7406, 7407, 7408, 7409, 7410, 7411, 7412, 7413, 7414, 7415, 7416, 7417, 7418, 7419, 7420, 7421, 7422, 7423, 7424, 7425, 7426, 7427, 7428, 7429, 7430, 7431, 7432, 7433, 7434, 7435, 7436, 7437, 7438, 7439, 7440, 7441, 7442, 7443, 7444, 7445, 7446, 7447, 7448, 7449, 7450, 7451, 7452, 7453, 7454, 7455, 7456, 7457, 7458, 7459, 7460, 7461, 7462, 7463, 7464, 7465, 7466, 7467, 7468, 7469, 7470, 7471, 7472, 7473, 7474, 7475, 7476, 7477, 7478, 7479, 7480, 7481, 7482, 7483, 7484, 7485, 7486, 7487,

In [27]:
count = 0
count_list = []
for i in spk2utt:
    count += len(spk2utt[i])
    count_list.append(len(spk2utt[i]))
print(count)

85764


In [28]:
with open(spk2utt_dict, 'wb') as handle:
    pickle.dump(spk2utt, handle)

with open(spk2utt_dict, 'rb') as handle:
    spk2utt = pickle.load(handle)
spk2utt_train = spk2utt

spk2utt_train = copy.deepcopy(spk2utt)

In [29]:
count = 0
count_list = []
for i in spk2utt_train:
    count += len(spk2utt_train[i])
    count_list.append(len(spk2utt[i]))
print(count)

85764


In [32]:
with open(spk2utt_train_dict, 'wb') as handle:
    pickle.dump(spk2utt_train, handle)

In [33]:
spk2utt_train_len = os.path.join(OPT_INDEX, 'spk2utt_train_len')

spk2utt_train_len_dict = {}

In [35]:
with open(spk2utt_train_dict, 'rb') as handle:
    spk2utt_train = pickle.load(handle)

for i in spk2utt_train:
    for utt in spk2utt_train[i]:
        spk2utt_train_len_dict[utt] = librosa.core.load(utt, sr=16000)[0].shape[0]

with open(spk2utt_train_len, 'wb') as handle:
    pickle.dump(spk2utt_train_len_dict, handle)

# Iterable Dataset & Make train data 488 small BASIC SET

In [3]:
from preprocessing import VoxIterableDataset

In [4]:
OPT_INDEX = '/Lun0/zhiyong/sdsvc_t2_train'
OPT_INDEX_1 = '/Lun0/zhiyong/dataset'
data_dir_dict = {}

# train
data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_train_dict')
data_dir_dict['music_dict'] = os.path.join(OPT_INDEX_1, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(OPT_INDEX_1, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(OPT_INDEX_1, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(OPT_INDEX_1, 'rir_dict')

In [5]:
data_len_dict = {}

data_len_dict['music_len'] = os.path.join(OPT_INDEX_1, 'music_len')
data_len_dict['noise_len'] = os.path.join(OPT_INDEX_1, 'noise_len')
data_len_dict['babble_len'] = os.path.join(OPT_INDEX_1, 'babble_len')
data_len_dict['spk2utt_train_len'] = os.path.join(OPT_INDEX, 'spk2utt_train_len')

In [6]:
OPT_INDEX = '/Lun0/zhiyong/sdsvc_t2_train'
with open(os.path.join(OPT_INDEX, 'spk2utt_train_dict'), 'rb') as handle:
    spk2utt_train_dict = pickle.load(handle)

In [7]:
spk2utt_train_dict.keys()

dict_keys([7323, 7324, 7325, 7326, 7327, 7328, 7329, 7330, 7331, 7332, 7333, 7334, 7335, 7336, 7337, 7338, 7339, 7340, 7341, 7342, 7343, 7344, 7345, 7346, 7347, 7348, 7349, 7350, 7351, 7352, 7353, 7354, 7355, 7356, 7357, 7358, 7359, 7360, 7361, 7362, 7363, 7364, 7365, 7366, 7367, 7368, 7369, 7370, 7371, 7372, 7373, 7374, 7375, 7376, 7377, 7378, 7379, 7380, 7381, 7382, 7383, 7384, 7385, 7386, 7387, 7388, 7389, 7390, 7391, 7392, 7393, 7394, 7395, 7396, 7397, 7398, 7399, 7400, 7401, 7402, 7403, 7404, 7405, 7406, 7407, 7408, 7409, 7410, 7411, 7412, 7413, 7414, 7415, 7416, 7417, 7418, 7419, 7420, 7421, 7422, 7423, 7424, 7425, 7426, 7427, 7428, 7429, 7430, 7431, 7432, 7433, 7434, 7435, 7436, 7437, 7438, 7439, 7440, 7441, 7442, 7443, 7444, 7445, 7446, 7447, 7448, 7449, 7450, 7451, 7452, 7453, 7454, 7455, 7456, 7457, 7458, 7459, 7460, 7461, 7462, 7463, 7464, 7465, 7466, 7467, 7468, 7469, 7470, 7471, 7472, 7473, 7474, 7475, 7476, 7477, 7478, 7479, 7480, 7481, 7482, 7483, 7484, 7485, 7486, 7487,

In [11]:
spk2utt_train_dict_small = {}
spk2utt_trial_dict_small = {}
for count, i in enumerate(spk2utt_train_dict):
    if count < 488:
        spk2utt_train_dict_small[1211+count] = spk2utt_train_dict[i]
    else:
        spk2utt_trial_dict_small[1211+count] = spk2utt_train_dict[i]

In [20]:
spk2utt_train_dict_small.keys()

dict_keys([1211, 1212, 1213, 1214, 1215, 1216, 1217, 1218, 1219, 1220, 1221, 1222, 1223, 1224, 1225, 1226, 1227, 1228, 1229, 1230, 1231, 1232, 1233, 1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241, 1242, 1243, 1244, 1245, 1246, 1247, 1248, 1249, 1250, 1251, 1252, 1253, 1254, 1255, 1256, 1257, 1258, 1259, 1260, 1261, 1262, 1263, 1264, 1265, 1266, 1267, 1268, 1269, 1270, 1271, 1272, 1273, 1274, 1275, 1276, 1277, 1278, 1279, 1280, 1281, 1282, 1283, 1284, 1285, 1286, 1287, 1288, 1289, 1290, 1291, 1292, 1293, 1294, 1295, 1296, 1297, 1298, 1299, 1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309, 1310, 1311, 1312, 1313, 1314, 1315, 1316, 1317, 1318, 1319, 1320, 1321, 1322, 1323, 1324, 1325, 1326, 1327, 1328, 1329, 1330, 1331, 1332, 1333, 1334, 1335, 1336, 1337, 1338, 1339, 1340, 1341, 1342, 1343, 1344, 1345, 1346, 1347, 1348, 1349, 1350, 1351, 1352, 1353, 1354, 1355, 1356, 1357, 1358, 1359, 1360, 1361, 1362, 1363, 1364, 1365, 1366, 1367, 1368, 1369, 1370, 1371, 1372, 1373, 1374, 1375,

In [21]:
OPT_INDEX = '/Lun0/zhiyong/sdsvc_t2_small_sparse'
with open(os.path.join(OPT_INDEX, 'spk2utt_train_dict_small'), 'wb') as handle:
    pickle.dump(spk2utt_train_dict_small, handle)
with open(os.path.join(OPT_INDEX, 'spk2utt_trial_dict_small'), 'wb') as handle:
    pickle.dump(spk2utt_trial_dict_small, handle)

In [10]:
OPT_INDEX = '/Lun0/zhiyong/sdsvc_t2_small_sparse'
with open(os.path.join(OPT_INDEX, 'spk2utt_train_dict_small'), 'rb') as handle:
    spk2utt_train_dict_small = pickle.load(handle)

In [56]:
spk2utt_train_dict_small_488 = spk2utt_train_dict_small
spk2utt_val_dict_small_488 = {}
for i in spk2utt_train_dict_small_488:
    spk2utt_val_dict_small_488[i] = [spk2utt_train_dict_small_488[i][0]]
    spk2utt_train_dict_small_488[i].pop(0)

In [57]:
spk2utt_val_dict_small_488[1211]

['/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101064.wav']

In [58]:
spk2utt_train_dict_small_488[1211][0]

'/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101065.wav'

In [59]:
OPT_INDEX = '/Lun0/zhiyong/sdsvc_t2_small_sparse'
with open(os.path.join(OPT_INDEX, 'spk2utt_train_dict_small_488'), 'wb') as handle:
    pickle.dump(spk2utt_train_dict_small_488, handle)
with open(os.path.join(OPT_INDEX, 'spk2utt_val_dict_small_488'), 'wb') as handle:
    pickle.dump(spk2utt_val_dict_small_488, handle)

In [60]:
OPT_INDEX = '/Lun0/zhiyong/sdsvc_t2_small_sparse'
OPT_INDEX_1 = '/Lun0/zhiyong/dataset'
OPT_INDEX_2 = '/Lun0/zhiyong/sdsvc_t2_train'
data_dir_dict = {}

# train
data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_train_dict_small_488')
data_dir_dict['music_dict'] = os.path.join(OPT_INDEX_1, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(OPT_INDEX_1, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(OPT_INDEX_1, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(OPT_INDEX_1, 'rir_dict')

In [61]:
data_len_dict = {}

data_len_dict['music_len'] = os.path.join(OPT_INDEX_1, 'music_len')
data_len_dict['noise_len'] = os.path.join(OPT_INDEX_1, 'noise_len')
data_len_dict['babble_len'] = os.path.join(OPT_INDEX_1, 'babble_len')
data_len_dict['spk2utt_train_len'] = os.path.join(OPT_INDEX_2, 'spk2utt_train_len')

In [62]:
config = {}

config['sr'] = 16000
config['repeats'] = 100
config['batch_size'] = 1
config['extended_prefectch'] = 1.0
config['out_dir'] = '/Lun0/zhiyong/sdsvc_t2_small_sparse/train_data_small_488/'
config['err_log'] = '/Lun0/zhiyong/sdsvc_t2_small_sparse/tmp_data_errlog_1'

def iter_data_preload(dataset, i):
    dataset.get_random_list()
    try:
        start_time = time.time()
        for count, (data, label) in enumerate(dataset):
            with open(config['out_dir']+str(i)+str('_')+str(count), 'wb') as handle:
                pickle.dump((data.astype(np.float16), label.astype(np.int16)), handle)
#             print(time.time()-start_time)
#             with open('/Lun0/zhiyong/sdsvc_t2_train/tmp_data_errlog_1', 'a') as f:
#                 f.write(str(time.time()-start_time)+'\n')
            start_time = time.time()
            if ((count+1)%10000) == 0:
                print((count+1)//10000)
    except Exception:
        with open(config['err_log'], 'a') as f:
            traceback.print_exc(file=f)
        raise Exception

dataset = VoxIterableDataset(data_dir_dict, data_len_dict, config)
# dataset.noise_data_preload2mem()
dataset.noise_data_preload()

processes = [Process(target = iter_data_preload, args = (dataset, i)) for i in range(2)]
start_time = time.time()
[p.start() for p in processes]
joined = [p.join() for p in processes]
print(time.time()-start_time)

preloading music_dict
100
200
300
400
500
600
preloading noise_dict
100
200
300
400
500
600
700
800
900
preloading babble_dict
100
200
300
400
1
1
2
2
3
3
4
4
2097.375262737274


# Make train index 488 small

In [63]:
488*200

97600

In [64]:
OPT_INDEX = '/Lun0/zhiyong/sdsvc_t2_small_sparse'
train_data_dir = os.path.join(OPT_INDEX, 'train_data_small_488')
expected_len = 97600
workers = 2
single_worker_len = int(expected_len / workers)
output = os.path.join(OPT_INDEX, 'train_data_small_488.csv')

In [65]:
assert expected_len == len(glob.glob(train_data_dir+'/*'))

In [66]:
count = 0
with open(output, 'w') as f:
    for i in range(workers):
        for j in range(single_worker_len):
            path = os.path.join(train_data_dir, str(i)+'_'+str(j))
            assert os.path.isfile(path)
            f.write(path+'\n')
            count += 1

In [67]:
count

97600

# Make Val 488 small

In [68]:
OPT_INDEX = '/Lun0/zhiyong/sdsvc_t2_small_sparse'
OPT_INDEX_1 = '/Lun0/zhiyong/dataset'
OPT_INDEX_2 = '/Lun0/zhiyong/sdsvc_t2_train'
data_dir_dict = {}

# val
data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_val_dict_small_488')
data_dir_dict['music_dict'] = os.path.join(OPT_INDEX_1, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(OPT_INDEX_1, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(OPT_INDEX_1, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(OPT_INDEX_1, 'rir_dict')

In [69]:
data_len_dict = {}


data_len_dict['music_len'] = os.path.join(OPT_INDEX_1, 'music_len')
data_len_dict['noise_len'] = os.path.join(OPT_INDEX_1, 'noise_len')
data_len_dict['babble_len'] = os.path.join(OPT_INDEX_1, 'babble_len')

data_len_dict['spk2utt_train_len'] = os.path.join(OPT_INDEX_2, 'spk2utt_train_len')

In [70]:
config = {}

config['sr'] = 16000
config['repeats'] = 3
config['batch_size'] = 1
config['extended_prefectch'] = 1.0
config['out_dir'] = '/Lun0/zhiyong/sdsvc_t2_small_sparse/val_data_small_488/'

def iter_data_preload(dataset, i):
    dataset.get_random_list()
    start_time = time.time()
    for count, (data, label) in enumerate(dataset):
        with open(config['out_dir']+str(i)+'_'+str(count), 'wb') as handle:
            pickle.dump((data.astype(np.float16), label.astype(np.int16)), handle)
        print(time.time()-start_time)
        start_time = time.time()

dataset = VoxIterableDataset(data_dir_dict, data_len_dict, config)
dataset.noise_data_preload()

processes = [Process(target = iter_data_preload, args = (dataset, i)) for i in range(1)]
start_time = time.time()
[p.start() for p in processes]
joined = [p.join() for p in processes]
print(time.time()-start_time)

preloading music_dict
100
200
300
400
500
600
preloading noise_dict
100
200
300
400
500
600
700
800
900
preloading babble_dict
100
200
300
400
0.11420488357543945
0.03443455696105957
0.05103898048400879
0.04207015037536621
0.0170135498046875
0.02994680404663086
0.035253286361694336
0.03775954246520996
0.09977030754089355
0.03185844421386719
0.06194138526916504
0.05798196792602539
0.025030136108398438
0.036322593688964844
0.02852034568786621
0.04640674591064453
0.04154825210571289
0.0879974365234375
0.043100833892822266
0.04963397979736328
0.035428524017333984
0.030287742614746094
0.07256817817687988
0.043370723724365234
0.027685880661010742
0.03412222862243652
0.035073280334472656
0.027054786682128906
0.02125263214111328
0.026624202728271484
0.02129054069519043
0.024415016174316406
0.015910863876342773
0.020914554595947266
0.03445863723754883
0.06809306144714355
0.04491090774536133
0.02565622329711914
0.023427963256835938
0.032621145248413086
0.024230003356933594
0.02620983123779297
0.

0.033739566802978516
0.044713497161865234
0.05485963821411133
0.03159070014953613
0.022808551788330078
0.0388035774230957
0.08774375915527344
0.08371305465698242
0.05856490135192871
0.03217458724975586
0.04185986518859863
0.07164144515991211
0.04651594161987305
0.02839064598083496
0.01775050163269043
0.014611005783081055
0.034224748611450195
0.027007102966308594
0.024057865142822266
0.03210330009460449
0.02120351791381836
0.01824355125427246
0.033158302307128906
0.0194399356842041
0.015850067138671875
0.042172908782958984
0.016104459762573242
0.023874998092651367
0.038594961166381836
0.06268620491027832
0.0262298583984375
0.028258085250854492
0.026607036590576172
0.04286789894104004
0.06821179389953613
0.033617496490478516
0.054206132888793945
0.0535123348236084
0.021448612213134766
0.06354236602783203
0.03166794776916504
0.030669212341308594
0.044220685958862305
0.017264127731323242
0.02553868293762207
0.035271644592285156
0.026501178741455078
0.04666566848754883
0.022950410842895508


0.03729891777038574
0.04379749298095703
0.07676482200622559
0.0507810115814209
0.03641319274902344
0.03915047645568848
0.059706926345825195
0.03799319267272949
0.030626296997070312
0.05303192138671875
0.0330042839050293
0.02462911605834961
0.019078969955444336
0.05323982238769531
0.05100083351135254
0.10239052772521973
0.02236008644104004
0.03138589859008789
0.08284282684326172
0.04597592353820801
0.0292055606842041
0.03840160369873047
0.04659247398376465
0.10736322402954102
0.052941083908081055
0.02790045738220215
0.027975797653198242
0.04343128204345703
0.028886795043945312
0.03727436065673828
0.020028352737426758
0.023766279220581055
0.04269218444824219
0.02005624771118164
0.03646063804626465
0.03486943244934082
0.03733348846435547
0.02559494972229004
0.03589463233947754
0.040308237075805664
0.0333096981048584
0.03348278999328613
0.07256817817687988
0.05769205093383789
0.020197629928588867
0.015157699584960938
0.018133163452148438
0.027301311492919922
0.022572755813598633
0.05029201

0.043830156326293945
0.016658544540405273
0.03923940658569336
0.038398027420043945
0.028325796127319336
0.034250497817993164
0.01874828338623047
0.022641897201538086
0.13335919380187988
0.020931243896484375
0.049712181091308594
0.05720210075378418
0.09067249298095703
0.027612924575805664
0.025897979736328125
0.04750800132751465
0.03837895393371582
0.026773452758789062
0.047679901123046875
0.03782296180725098
0.03485393524169922
0.06368160247802734
0.049143314361572266
0.04413604736328125
0.02434992790222168
0.048070669174194336
0.1082770824432373
0.09962725639343262
0.028192520141601562
0.019127845764160156
0.04154014587402344
0.027363061904907227
0.019076824188232422
0.02463507652282715
0.027556896209716797
0.0642387866973877
0.0550534725189209
0.017641782760620117
0.03169417381286621
0.048126220703125
0.027137041091918945
0.033447980880737305
0.05440568923950195
0.032961130142211914
0.02701091766357422
0.02874135971069336
0.030230283737182617
0.024991512298583984
0.03740978240966797


# Make validation index 488 small

In [71]:
OPT_INDEX = '/Lun0/zhiyong/sdsvc_t2_small_sparse'
train_data_dir = os.path.join(OPT_INDEX, 'val_data_small_488')
expected_len = 488 * 3
workers = 1
single_worker_len = int(expected_len / workers)
output = os.path.join(OPT_INDEX, 'val_data_small_488.csv')

In [72]:
assert expected_len == len(glob.glob(train_data_dir+'/*'))

In [73]:
count = 0
with open(output, 'w') as f:
    for i in range(workers):
        for j in range(single_worker_len):
            path = os.path.join(train_data_dir, str(i)+'_'+str(j))
            assert os.path.isfile(path)
            f.write(path+'\n')
            count += 1

In [74]:
count

1464

# make trial

In [34]:
with open(os.path.join(OPT_INDEX, 'spk2utt_trial_dict_small'), 'rb') as handle:
    spk2utt_trial_dict_small = pickle.load(handle)

In [42]:
count = 0
for i in spk2utt_trial_dict_small:
    count += len(spk2utt_trial_dict_small[i])
print(count)

14490


In [23]:
OPT_INDEX_1 = '/Lun0/zhiyong/dataset'
data_dir_dict = {}

# train
data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_train_dict')
data_dir_dict['music_dict'] = os.path.join(OPT_INDEX_1, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(OPT_INDEX_1, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(OPT_INDEX_1, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(OPT_INDEX_1, 'rir_dict')

In [24]:
data_len_dict = {}

data_len_dict['music_len'] = os.path.join(OPT_INDEX_1, 'music_len')
data_len_dict['noise_len'] = os.path.join(OPT_INDEX_1, 'noise_len')
data_len_dict['babble_len'] = os.path.join(OPT_INDEX_1, 'babble_len')
data_len_dict['spk2utt_train_len'] = os.path.join(OPT_INDEX, 'spk2utt_train_len')
# data_len_dict['spk2utt_val_len'] = os.path.join(OPT_INDEX, 'spk2utt_val_len')

In [28]:
config = {}

config['sr'] = 16000
config['repeats'] = None
config['batch_size'] = None
config['extended_prefectch'] = None
config['out_dir'] = '/Lun0/zhiyong/sdsvc_t2_train/trial_data_small/'

trial_dict_dir = os.path.join(OPT_INDEX, 'spk2utt_trial_dict_small')

def trial_data_preload(dataset, i, trial_dict_dir):
    with open(trial_dict_dir, 'rb') as handle:
        trial_dict = pickle.load(handle)
        
    start_time = time.time()
    count = 0
    for j in trial_dict:
        for line in trial_dict[j]:
            data = dataset.process_one_utt(line)
            label = line.split('/')[-1][:-4]
            with open(config['out_dir']+str(i)+'_'+str(count), 'wb') as handle:
                pickle.dump((data.astype(np.float16), [label]), handle)
            print(time.time()-start_time)
            start_time = time.time()
            count += 1

dataset = VoxIterableDataset(data_dir_dict, data_len_dict, config)

processes = [Process(target = trial_data_preload, args = (dataset, i, trial_dict_dir)) for i in range(1)]
start_time = time.time()
[p.start() for p in processes]
joined = [p.join() for p in processes]
print(time.time()-start_time)

7.970787048339844
0.09568214416503906
0.07162857055664062
0.04517817497253418
0.11772346496582031
0.026267051696777344
0.23095703125
0.39521265029907227
0.30336642265319824
0.1292102336883545
0.3331153392791748
0.19955086708068848
0.10549378395080566
0.07919764518737793
0.10640358924865723
0.3774123191833496
0.19918084144592285
0.2307279109954834
0.19574284553527832
0.3441331386566162
0.16498899459838867
0.14908194541931152
0.19398808479309082
0.24889254570007324
0.10834956169128418
0.0637972354888916
0.0650031566619873
0.16168951988220215
0.08496427536010742
0.07648086547851562
0.09900832176208496
0.02625560760498047
0.08376407623291016
0.1414504051208496
0.057369232177734375
0.040392160415649414
0.1705338954925537
0.7836112976074219
0.2490849494934082
0.08530545234680176
0.21913695335388184
0.5410234928131104
0.2864954471588135
0.11587405204772949
0.08620929718017578
0.26518964767456055
0.032135009765625
0.09271383285522461
0.21153616905212402
0.08006024360656738
0.11550784111022949


0.007974624633789062
0.05493474006652832
0.0890343189239502
0.05025672912597656
0.03162360191345215
0.1323080062866211
0.028889894485473633
0.023050785064697266
0.13179278373718262
0.1176598072052002
0.0513918399810791
0.01606154441833496
0.046758413314819336
0.046460628509521484
0.03598284721374512
0.11268186569213867
0.019454479217529297
0.022535324096679688
0.07387661933898926
0.0386500358581543
0.03748798370361328
0.030466318130493164
0.04042410850524902
0.030204057693481445
0.1830606460571289
0.028326988220214844
0.17831969261169434
0.06718063354492188
0.05417346954345703
0.027420520782470703
0.02719712257385254
0.013175010681152344
0.03625988960266113
0.02642989158630371
0.012691974639892578
0.014596939086914062
0.04148435592651367
0.03883934020996094
0.04359769821166992
0.01464533805847168
0.0478215217590332
0.0482785701751709
0.04378199577331543
0.04619145393371582
0.0114898681640625
0.028397321701049805
0.02158069610595703
0.08585548400878906
0.015172958374023438
0.03850936889

0.04865312576293945
0.02405381202697754
0.0451817512512207
0.026067495346069336
0.0327911376953125
0.04004049301147461
0.04676628112792969
0.025220870971679688
0.03541421890258789
0.030923128128051758
0.03136277198791504
0.019239187240600586
0.014071226119995117
0.021603107452392578
0.017856121063232422
0.04097414016723633
0.0434415340423584
0.051842689514160156
0.023006200790405273
0.041776180267333984
0.028171300888061523
0.022443056106567383
0.027190208435058594
0.07228493690490723
0.030019044876098633
0.01363062858581543
0.03919816017150879
0.025595903396606445
0.04029726982116699
0.03909635543823242
0.021361589431762695
0.02166891098022461
0.03622579574584961
0.029955148696899414
0.020288944244384766
0.02585744857788086
0.04838705062866211
0.05249953269958496
0.02307415008544922
0.03559446334838867
0.033498525619506836
0.01413726806640625
0.018524646759033203
0.018862009048461914
0.023082256317138672
0.02685260772705078
0.028162479400634766
0.023384571075439453
0.03879308700561523

0.035897016525268555
0.025051593780517578
0.027057886123657227
0.039173126220703125
0.04752182960510254
0.04201769828796387
0.040871381759643555
0.05004763603210449
0.018031835556030273
0.020064115524291992
0.06719589233398438
0.0245974063873291
0.008975982666015625
0.04356074333190918
0.037656545639038086
0.02707195281982422
0.04280400276184082
0.0424647331237793
0.029360532760620117
0.04414248466491699
0.04583168029785156
0.06044363975524902
0.05581951141357422
0.04149627685546875
0.013538122177124023
0.04976940155029297
0.021921634674072266
0.03431439399719238
0.06729364395141602
0.08485984802246094
0.04916262626647949
0.08037400245666504
0.012993574142456055
0.027746915817260742
0.06945562362670898
0.035152435302734375
0.1828312873840332
0.01967644691467285
0.04039764404296875
0.032108306884765625
0.024156570434570312
0.03089284896850586
0.026799440383911133
0.03312492370605469
0.021024465560913086
0.06476092338562012
0.009063005447387695
0.021511077880859375
0.00863957405090332
0.

2.1972687244415283
1.3971798419952393
1.313539981842041
1.739635705947876
4.108907699584961
0.467421293258667
0.9667513370513916
0.05855107307434082
0.28678274154663086
1.97163987159729
0.46239781379699707
0.07429027557373047
0.36092090606689453
0.2646481990814209
0.03656268119812012
0.2998044490814209
0.7773001194000244
0.8321676254272461
0.8578882217407227
0.009168386459350586
0.043726444244384766
0.09692120552062988
0.054459333419799805
0.13554048538208008
0.1656644344329834
2.8685834407806396
0.41100049018859863
0.026394128799438477
0.263840913772583
1.157127857208252
0.06248354911804199
1.1873576641082764
0.4274613857269287
0.1032097339630127
0.4692668914794922
0.170518159866333
0.29334211349487305
0.25072288513183594
0.19213414192199707
0.03535056114196777
0.010788440704345703
0.02080082893371582
0.02030348777770996
0.5120110511779785
0.02980780601501465
1.9336097240447998
1.0281181335449219
0.1697683334350586
1.2153441905975342
0.713829755783081
0.3317556381225586
0.223875045776

...

0.037073612213134766
0.013927698135375977
0.016885757446289062
0.017002582550048828
0.02785491943359375
0.03777575492858887
0.023759841918945312
0.01916337013244629
0.03456568717956543
0.013216257095336914
0.02713322639465332
0.02323150634765625
0.02130866050720215
0.01797175407409668
0.012658834457397461
0.018262147903442383
0.026245594024658203
0.013047456741333008
0.03781938552856445
0.011188030242919922
0.017562150955200195
0.02629995346069336
0.029883623123168945
0.019441604614257812
0.023482799530029297
0.02915787696838379
0.018626689910888672
0.02840256690979004
0.0808866024017334
0.017884254455566406
0.039913177490234375
0.03452324867248535
0.05104207992553711
0.0314178466796875
0.024317026138305664
0.11610770225524902
0.0536196231842041
0.016313791275024414
0.055240631103515625
0.012454032897949219
0.048119306564331055
0.05050206184387207
0.055235862731933594
0.04654264450073242
0.01653432846069336
0.029491901397705078
0.013106107711791992
0.025950193405151367
0.03830099105834

0.01605391502380371
0.07575511932373047
0.06116008758544922
0.024808168411254883
0.02361917495727539
0.022581100463867188
0.02937030792236328
0.04920053482055664
0.04553484916687012
0.01332712173461914
0.017982959747314453
0.04599142074584961
0.013501644134521484
0.008922815322875977
0.04625105857849121
0.012485265731811523
0.0448145866394043
0.07291030883789062
0.04283785820007324
0.02655768394470215
0.011575937271118164
0.03184866905212402
0.06090950965881348
0.013205528259277344
0.036824941635131836
0.06027340888977051
0.03470492362976074
0.022298812866210938
0.021738052368164062
0.021153926849365234
0.04631662368774414
0.02683877944946289
0.03580737113952637
0.01482248306274414
0.0281374454498291
0.04406857490539551
0.05017518997192383
0.020368576049804688
0.015338420867919922
0.0386204719543457
0.06636524200439453
0.03815746307373047
0.01965928077697754
0.019710063934326172
0.025119543075561523
0.04252433776855469
0.04407811164855957
0.027927398681640625
0.01834583282470703
0.0470

1.0485117435455322
1.2707335948944092
1.9963328838348389
0.6359927654266357
2.182634115219116
3.523883581161499
2.2113234996795654
0.8926291465759277
0.8202190399169922
1.0509560108184814
1.4194645881652832
1.1529457569122314
0.9809703826904297
0.8643782138824463
1.1796085834503174
0.05280137062072754
2.030210018157959
0.4960193634033203
0.07272005081176758
2.0743699073791504
3.013974666595459
1.2114081382751465
4.370327711105347
1.1037144660949707
1.048722505569458
1.1147353649139404
1.1138031482696533
1.087616205215454
1.0201284885406494
1.0942268371582031
1.106203317642212
1.15787935256958
1.1022720336914062
1.0616469383239746
1.1633899211883545
1.1726670265197754
0.38679957389831543
0.7927103042602539
1.7586607933044434
0.05429840087890625
0.020880699157714844
0.11966466903686523
0.2434554100036621
1.1960184574127197
0.12302899360656738
0.5797855854034424
0.3417956829071045
2.1145527362823486
1.2613787651062012
1.1569430828094482
1.7336621284484863
0.2468867301940918
0.012490272521

0.15289855003356934
0.7253079414367676
0.9346864223480225
0.32865428924560547
0.9996953010559082
0.3559451103210449
1.3267796039581299
0.43857765197753906
0.1033322811126709
1.0116751194000244
0.3666675090789795
0.3511781692504883
1.4935805797576904
0.5397946834564209
0.03809475898742676
2.0265092849731445
0.384537935256958
0.028731822967529297
0.27829432487487793
0.09460020065307617
0.5935139656066895
1.606943130493164
0.10349011421203613
0.5794134140014648
0.2139899730682373
0.19613146781921387
0.058611154556274414
0.2962160110473633
0.7471151351928711
1.7822701930999756
1.9501569271087646
1.5906496047973633
1.9475831985473633
1.3644273281097412
1.643341064453125
1.4346380233764648
0.3211967945098877
0.5898559093475342
0.072235107421875
1.7195866107940674
0.11302065849304199
0.1666567325592041
2.672945499420166
0.16869139671325684
2.0858850479125977
1.9773976802825928
1.7474818229675293
1.1302778720855713
1.5498123168945312
0.23997259140014648
2.1516964435577393
1.1555042266845703
2.

0.12375569343566895
0.38872671127319336
0.1963198184967041
0.9333386421203613
1.2721211910247803
1.0014073848724365
1.2887580394744873
0.05254793167114258
0.41472411155700684
0.19922089576721191
0.03579854965209961
0.24574947357177734
1.6593806743621826
0.38623690605163574
0.22296905517578125
0.3862273693084717
2.4754741191864014
0.0694434642791748
0.8597302436828613
1.1858983039855957
0.11170458793640137
1.7373418807983398
0.526630163192749
1.0522677898406982
0.7010986804962158
0.22445917129516602
0.5351536273956299
0.4286079406738281
0.07278037071228027
0.37714338302612305
0.19305181503295898
2.1805717945098877
0.22693705558776855
1.0201926231384277
1.0115714073181152
0.3616964817047119
0.017688274383544922
1.0513427257537842
1.1298346519470215
1.0794332027435303
0.4036116600036621
0.01569223403930664
0.7697319984436035
1.9962794780731201
0.08423638343811035
0.19418096542358398
2.3613030910491943
0.7790038585662842
2.240300416946411
1.7471272945404053
0.9240307807922363
0.40735745429

# make trial index

In [43]:
# OPT_INDEX = '/Lun0/zhiyong/dataset'
train_data_dir = os.path.join(OPT_INDEX, 'trial_data_small')
expected_len = 14490
workers = 1
single_worker_len = int(expected_len / workers)
output = os.path.join(OPT_INDEX, 'trial_data_small_csv.csv')

In [44]:
len(glob.glob(train_data_dir+'/*'))

14490

In [45]:
assert expected_len == len(glob.glob(train_data_dir+'/*'))

In [47]:
count = 0
with open(output, 'w') as f:
    for i in range(workers):
        for j in range(single_worker_len):
            path = os.path.join(train_data_dir, str(i)+'_'+str(j))
            assert os.path.isfile(path)
            f.write(path+'\n')
            count += 1

In [48]:
count

14490

# make trial list

In [59]:
trial_list_out_dir = os.path.join(OPT_INDEX, 'small_tiral_list')
trial_num_per_utt = 8

In [58]:
with open(os.path.join(OPT_INDEX, 'spk2utt_trial_dict_small'), 'rb') as handle:
    spk2utt_trial_dict_small = pickle.load(handle)

In [119]:
count = 0
with open(trial_list_out_dir, 'w') as of:
    for i in spk2utt_trial_dict_small:
        for line in spk2utt_trial_dict_small[i]:
            for c in range(trial_num_per_utt):
                if (c % 2) == 0:
                    tmp_list = copy.deepcopy(spk2utt_trial_dict_small[i])
                    tmp_list.remove(line)
                    label_t = random.choice(tmp_list)
                    label_t = label_t.split('/')[-1]
                    label_e = line.split('/')[-1]
                    of.write(label_e+' '+label_t+' '+'target'+'\n')
                    count += 1
                else:
                    tmp_list = list(spk2utt_trial_dict_small.keys())
                    tmp_list.remove(i)
                    spk_t = random.choice(tmp_list)
                    label_t = random.choice(spk2utt_trial_dict_small[spk_t])
                    label_t = label_t.split('/')[-1]
                    label_e = line.split('/')[-1]
                    of.write(label_e+' '+label_t+' '+'nontarget'+'\n')                    
                    count += 1
print(count)

115920


# make small 100 train

In [22]:
OPT_INDEX = '/Lun0/zhiyong/sdsvc_t2_small_sparse'
OPT_INDEX_1 = '/Lun0/zhiyong/dataset'
OPT_INDEX_2 = '/Lun0/zhiyong/sdsvc_t2_train'
data_dir_dict = {}

# train
# data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_train_dict')
data_dir_dict['music_dict'] = os.path.join(OPT_INDEX_1, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(OPT_INDEX_1, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(OPT_INDEX_1, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(OPT_INDEX_1, 'rir_dict')

In [23]:
data_len_dict = {}

data_len_dict['music_len'] = os.path.join(OPT_INDEX_1, 'music_len')
data_len_dict['noise_len'] = os.path.join(OPT_INDEX_1, 'noise_len')
data_len_dict['babble_len'] = os.path.join(OPT_INDEX_1, 'babble_len')
data_len_dict['spk2utt_train_len'] = os.path.join(OPT_INDEX_2, 'spk2utt_train_len')
# data_len_dict['spk2utt_val_len'] = os.path.join(OPT_INDEX, 'spk2utt_val_len')

In [5]:
with open(os.path.join(OPT_INDEX, 'spk2utt_train_dict_small'), 'rb') as handle:
    spk2utt_train_dict = pickle.load(handle)

In [6]:
spk2utt_train_dict.keys()

dict_keys([1211, 1212, 1213, 1214, 1215, 1216, 1217, 1218, 1219, 1220, 1221, 1222, 1223, 1224, 1225, 1226, 1227, 1228, 1229, 1230, 1231, 1232, 1233, 1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241, 1242, 1243, 1244, 1245, 1246, 1247, 1248, 1249, 1250, 1251, 1252, 1253, 1254, 1255, 1256, 1257, 1258, 1259, 1260, 1261, 1262, 1263, 1264, 1265, 1266, 1267, 1268, 1269, 1270, 1271, 1272, 1273, 1274, 1275, 1276, 1277, 1278, 1279, 1280, 1281, 1282, 1283, 1284, 1285, 1286, 1287, 1288, 1289, 1290, 1291, 1292, 1293, 1294, 1295, 1296, 1297, 1298, 1299, 1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309, 1310, 1311, 1312, 1313, 1314, 1315, 1316, 1317, 1318, 1319, 1320, 1321, 1322, 1323, 1324, 1325, 1326, 1327, 1328, 1329, 1330, 1331, 1332, 1333, 1334, 1335, 1336, 1337, 1338, 1339, 1340, 1341, 1342, 1343, 1344, 1345, 1346, 1347, 1348, 1349, 1350, 1351, 1352, 1353, 1354, 1355, 1356, 1357, 1358, 1359, 1360, 1361, 1362, 1363, 1364, 1365, 1366, 1367, 1368, 1369, 1370, 1371, 1372, 1373, 1374, 1375,

In [11]:
spk2utt_train_dict_small_100 = {}
spk2utt_val_dict_small_100 = {}
for count, i in enumerate(spk2utt_train_dict):
    if count < 100:
        spk2utt_train_dict_small_100[1211+count] = spk2utt_train_dict[i]
    else:
        break

In [12]:
spk2utt_train_dict_small_100.keys()

dict_keys([1211, 1212, 1213, 1214, 1215, 1216, 1217, 1218, 1219, 1220, 1221, 1222, 1223, 1224, 1225, 1226, 1227, 1228, 1229, 1230, 1231, 1232, 1233, 1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241, 1242, 1243, 1244, 1245, 1246, 1247, 1248, 1249, 1250, 1251, 1252, 1253, 1254, 1255, 1256, 1257, 1258, 1259, 1260, 1261, 1262, 1263, 1264, 1265, 1266, 1267, 1268, 1269, 1270, 1271, 1272, 1273, 1274, 1275, 1276, 1277, 1278, 1279, 1280, 1281, 1282, 1283, 1284, 1285, 1286, 1287, 1288, 1289, 1290, 1291, 1292, 1293, 1294, 1295, 1296, 1297, 1298, 1299, 1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309, 1310])

In [13]:
spk2utt_train_dict_small_100[1211]

['/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101064.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101065.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101066.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101067.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101068.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101069.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101070.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101071.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101072.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101073.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101074.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101075.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101076.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/tas

In [14]:
for i in spk2utt_train_dict_small_100:
    spk2utt_val_dict_small_100[i] = [spk2utt_train_dict_small_100[i][0]]
    spk2utt_train_dict_small_100[i].pop(0)

In [16]:
spk2utt_train_dict_small_100[1211]

['/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101065.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101066.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101067.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101068.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101069.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101070.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101071.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101072.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101073.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101074.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101075.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101076.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101077.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/tas

In [17]:
spk2utt_val_dict_small_100[1211]

['/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101064.wav']

In [18]:
with open(os.path.join(OPT_INDEX, 'spk2utt_train_dict_small_100'), 'wb') as handle:
    pickle.dump(spk2utt_train_dict_small_100, handle)
with open(os.path.join(OPT_INDEX, 'spk2utt_val_dict_small_100'), 'wb') as handle:
    pickle.dump(spk2utt_val_dict_small_100, handle)

In [25]:
data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_train_dict_small_100')

In [20]:
100 * 200

20000

In [26]:
config = {}

config['sr'] = 16000
config['repeats'] = 100
config['batch_size'] = 1
config['extended_prefectch'] = 1.0
config['out_dir'] = '/Lun0/zhiyong/sdsvc_t2_small_sparse/train_data_small_100/'
config['err_log'] = '/Lun0/zhiyong/sdsvc_t2_small_sparse/tmp_data_errlog_1'

def iter_data_preload(dataset, i):
    dataset.get_random_list()
    try:
        start_time = time.time()
        for count, (data, label) in enumerate(dataset):
            with open(config['out_dir']+str(i)+str('_')+str(count), 'wb') as handle:
                pickle.dump((data.astype(np.float16), label.astype(np.int16)), handle)
#             print(time.time()-start_time)
            with open(config['err_log'], 'a') as f:
                f.write(str(time.time()-start_time)+'\n')
            start_time = time.time()
            if ((count+1)%10000) == 0:
                print((count+1)//10000)
    except Exception:
        with open(config['err_log'], 'a') as f:
            traceback.print_exc(file=f)
        raise Exception

dataset = VoxIterableDataset(data_dir_dict, data_len_dict, config)
# dataset.noise_data_preload2mem()
dataset.noise_data_preload()

processes = [Process(target = iter_data_preload, args = (dataset, i)) for i in range(2)]
start_time = time.time()
[p.start() for p in processes]
joined = [p.join() for p in processes]
print(time.time()-start_time)

preloading music_dict
100
200
300
400
500
600
preloading noise_dict
100
200
300
400
500
600
700
800
900
preloading babble_dict
100
200
300
400
1
1
915.2412629127502


# Make train index

In [27]:
100*200

20000

In [28]:
# OPT_INDEX = '/Lun0/zhiyong/dataset'
train_data_dir = os.path.join(OPT_INDEX, 'train_data_small_100')
expected_len = 20000
workers = 2
single_worker_len = int(expected_len / workers)
output = os.path.join(OPT_INDEX, 'train_data_small_100.csv')

In [29]:
assert expected_len == len(glob.glob(train_data_dir+'/*'))

In [30]:
count = 0
with open(output, 'w') as f:
    for i in range(workers):
        for j in range(single_worker_len):
            path = os.path.join(train_data_dir, str(i)+'_'+str(j))
            assert os.path.isfile(path)
            f.write(path+'\n')
            count += 1

In [31]:
count

20000

# Make val data

In [43]:
OPT_INDEX = '/Lun0/zhiyong/sdsvc_t2_small_sparse'
OPT_INDEX_1 = '/Lun0/zhiyong/dataset'
OPT_INDEX_2 = '/Lun0/zhiyong/sdsvc_t2_train'
data_dir_dict = {}

# val
data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_val_dict_small_100')
data_dir_dict['music_dict'] = os.path.join(OPT_INDEX_1, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(OPT_INDEX_1, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(OPT_INDEX_1, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(OPT_INDEX_1, 'rir_dict')

In [44]:
data_len_dict = {}


data_len_dict['music_len'] = os.path.join(OPT_INDEX_1, 'music_len')
data_len_dict['noise_len'] = os.path.join(OPT_INDEX_1, 'noise_len')
data_len_dict['babble_len'] = os.path.join(OPT_INDEX_1, 'babble_len')

data_len_dict['spk2utt_train_len'] = os.path.join(OPT_INDEX_2, 'spk2utt_train_len')

In [45]:
config = {}

config['sr'] = 16000
config['repeats'] = 10
config['batch_size'] = 1
config['extended_prefectch'] = 1.0
config['out_dir'] = '/Lun0/zhiyong/sdsvc_t2_small_sparse/val_data_small_100/'

def iter_data_preload(dataset, i):
    dataset.get_random_list()
    start_time = time.time()
    for count, (data, label) in enumerate(dataset):
        with open(config['out_dir']+str(i)+'_'+str(count), 'wb') as handle:
            pickle.dump((data.astype(np.float16), label.astype(np.int16)), handle)
        print(time.time()-start_time)
        start_time = time.time()

dataset = VoxIterableDataset(data_dir_dict, data_len_dict, config)
dataset.noise_data_preload()

processes = [Process(target = iter_data_preload, args = (dataset, i)) for i in range(1)]
start_time = time.time()
[p.start() for p in processes]
joined = [p.join() for p in processes]
print(time.time()-start_time)

preloading music_dict
100
200
300
400
500
600
preloading noise_dict
100
200
300
400
500
600
700
800
900
preloading babble_dict
100
200
300
400
0.0607149600982666
0.023008108139038086
0.01995849609375
0.036695003509521484
0.0258028507232666
0.016278743743896484
0.0295562744140625
0.027189254760742188
0.021027326583862305
0.0285341739654541
0.01634836196899414
0.25923919677734375
0.01499795913696289
0.014767885208129883
0.022329092025756836
0.31005096435546875
0.04580855369567871
0.014839649200439453
0.08670830726623535
0.02222609519958496
0.023081064224243164
0.016374826431274414
0.020233631134033203
0.02009892463684082
0.04083967208862305
0.04007530212402344
0.018540620803833008
0.032558441162109375
0.02991771697998047
0.013693809509277344
0.016008853912353516
0.06458449363708496
0.024743080139160156
0.018505096435546875
0.024322986602783203
0.027758359909057617
0.02572154998779297
0.04434561729431152
0.02620983123779297
0.04402875900268555
0.0653080940246582
0.024740934371948242
0.028

0.034413814544677734
0.023209810256958008
0.013323307037353516
0.013197183609008789
0.029984235763549805
0.012975215911865234
0.01969623565673828
0.015119791030883789
0.023197412490844727
0.01857614517211914
0.023251771926879883
0.016325712203979492
0.014032840728759766
0.012980222702026367
0.016449928283691406
0.020786762237548828
0.014128446578979492
0.014104127883911133
0.014223098754882812
0.022285938262939453
0.021449804306030273
0.02559638023376465
0.025311946868896484
0.019215106964111328
0.01591038703918457
0.02178788185119629
0.027680635452270508
0.015540361404418945
0.017466306686401367
0.02181553840637207
0.010376930236816406
0.013589859008789062
0.028358936309814453
0.030715465545654297
0.021480321884155273
0.01503753662109375
0.03049635887145996
0.027118682861328125
0.015598297119140625
0.0321505069732666
0.04257345199584961
0.040277957916259766
0.03894329071044922
0.029438018798828125
0.037130117416381836
0.032605648040771484
0.017338275909423828
0.025855302810668945
0.01

0.031209230422973633
0.04420137405395508
0.024485111236572266
0.02309560775756836
0.02918839454650879
0.02337813377380371
0.024019241333007812
0.024120807647705078
0.025516033172607422
0.011083126068115234
0.011188030242919922
0.024323225021362305
0.03825569152832031
0.02186417579650879
0.029891490936279297
0.029332876205444336
0.04009294509887695
0.04211854934692383
0.033449411392211914
0.06770038604736328
0.01568007469177246
0.04836773872375488
0.02827167510986328
0.026676654815673828
0.016595125198364258
0.014835596084594727
0.03053736686706543
0.02290511131286621
0.020047426223754883
0.014983654022216797
0.020430564880371094
0.02240157127380371
0.030425310134887695
0.01589798927307129
0.03205275535583496
0.014985799789428711
0.02791118621826172
0.010134696960449219
0.01834392547607422
0.012473344802856445
0.03253626823425293
0.02191329002380371
0.025969266891479492
0.012841939926147461
0.025908231735229492
0.014864921569824219
0.02625298500061035
0.01291036605834961
0.0125393867492

# Make validation index (Need only once)

In [46]:
OPT_INDEX = '/Lun0/zhiyong/sdsvc_t2_small_sparse'
train_data_dir = os.path.join(OPT_INDEX, 'val_data_small_100')
expected_len = 100 * 10
workers = 1
single_worker_len = int(expected_len / workers)
output = os.path.join(OPT_INDEX, 'val_data_small_100.csv')

In [47]:
assert expected_len == len(glob.glob(train_data_dir+'/*'))

In [48]:
count = 0
with open(output, 'w') as f:
    for i in range(workers):
        for j in range(single_worker_len):
            path = os.path.join(train_data_dir, str(i)+'_'+str(j))
            assert os.path.isfile(path)
            f.write(path+'\n')
            count += 1

In [49]:
count

1000

# make small 50 train

In [12]:
OPT_INDEX = '/Lun0/zhiyong/sdsvc_t2_small_sparse'
OPT_INDEX_1 = '/Lun0/zhiyong/dataset'
OPT_INDEX_2 = '/Lun0/zhiyong/sdsvc_t2_train'
data_dir_dict = {}

# train
# data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_train_dict')
data_dir_dict['music_dict'] = os.path.join(OPT_INDEX_1, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(OPT_INDEX_1, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(OPT_INDEX_1, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(OPT_INDEX_1, 'rir_dict')

In [13]:
data_len_dict = {}

data_len_dict['music_len'] = os.path.join(OPT_INDEX_1, 'music_len')
data_len_dict['noise_len'] = os.path.join(OPT_INDEX_1, 'noise_len')
data_len_dict['babble_len'] = os.path.join(OPT_INDEX_1, 'babble_len')
data_len_dict['spk2utt_train_len'] = os.path.join(OPT_INDEX_2, 'spk2utt_train_len')
# data_len_dict['spk2utt_val_len'] = os.path.join(OPT_INDEX, 'spk2utt_val_len')

In [14]:
with open(os.path.join(OPT_INDEX, 'spk2utt_train_dict_small'), 'rb') as handle:
    spk2utt_train_dict = pickle.load(handle)

In [15]:
spk2utt_train_dict.keys()

dict_keys([1211, 1212, 1213, 1214, 1215, 1216, 1217, 1218, 1219, 1220, 1221, 1222, 1223, 1224, 1225, 1226, 1227, 1228, 1229, 1230, 1231, 1232, 1233, 1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241, 1242, 1243, 1244, 1245, 1246, 1247, 1248, 1249, 1250, 1251, 1252, 1253, 1254, 1255, 1256, 1257, 1258, 1259, 1260, 1261, 1262, 1263, 1264, 1265, 1266, 1267, 1268, 1269, 1270, 1271, 1272, 1273, 1274, 1275, 1276, 1277, 1278, 1279, 1280, 1281, 1282, 1283, 1284, 1285, 1286, 1287, 1288, 1289, 1290, 1291, 1292, 1293, 1294, 1295, 1296, 1297, 1298, 1299, 1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309, 1310, 1311, 1312, 1313, 1314, 1315, 1316, 1317, 1318, 1319, 1320, 1321, 1322, 1323, 1324, 1325, 1326, 1327, 1328, 1329, 1330, 1331, 1332, 1333, 1334, 1335, 1336, 1337, 1338, 1339, 1340, 1341, 1342, 1343, 1344, 1345, 1346, 1347, 1348, 1349, 1350, 1351, 1352, 1353, 1354, 1355, 1356, 1357, 1358, 1359, 1360, 1361, 1362, 1363, 1364, 1365, 1366, 1367, 1368, 1369, 1370, 1371, 1372, 1373, 1374, 1375,

In [17]:
spk2utt_train_dict_small_50 = {}
spk2utt_val_dict_small_50 = {}
for count, i in enumerate(spk2utt_train_dict):
    if count < 50:
        spk2utt_train_dict_small_50[1211+count] = spk2utt_train_dict[i]
    else:
        break

In [18]:
spk2utt_train_dict_small_50.keys()

dict_keys([1211, 1212, 1213, 1214, 1215, 1216, 1217, 1218, 1219, 1220, 1221, 1222, 1223, 1224, 1225, 1226, 1227, 1228, 1229, 1230, 1231, 1232, 1233, 1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241, 1242, 1243, 1244, 1245, 1246, 1247, 1248, 1249, 1250, 1251, 1252, 1253, 1254, 1255, 1256, 1257, 1258, 1259, 1260])

In [19]:
spk2utt_train_dict_small_50[1211]

['/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101064.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101065.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101066.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101067.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101068.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101069.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101070.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101071.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101072.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101073.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101074.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101075.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101076.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/tas

In [20]:
for i in spk2utt_train_dict_small_50:
    spk2utt_val_dict_small_50[i] = [spk2utt_train_dict_small_50[i][0]]
    spk2utt_train_dict_small_50[i].pop(0)

In [21]:
spk2utt_train_dict_small_50[1211]

['/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101065.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101066.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101067.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101068.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101069.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101070.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101071.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101072.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101073.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101074.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101075.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101076.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101077.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/tas

In [22]:
spk2utt_val_dict_small_50[1211]

['/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101064.wav']

In [44]:
with open(os.path.join(OPT_INDEX, 'spk2utt_train_dict_small_50'), 'wb') as handle:
    pickle.dump(spk2utt_train_dict_small_50, handle)
with open(os.path.join(OPT_INDEX, 'spk2utt_val_dict_small_50'), 'wb') as handle:
    pickle.dump(spk2utt_val_dict_small_50, handle)

In [24]:
data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_train_dict_small_50')

In [25]:
50 * 200

10000

In [26]:
config = {}

config['sr'] = 16000
config['repeats'] = 100
config['batch_size'] = 1
config['extended_prefectch'] = 1.0
config['out_dir'] = '/Lun0/zhiyong/sdsvc_t2_small_sparse/train_data_small_50/'
config['err_log'] = '/Lun0/zhiyong/sdsvc_t2_small_sparse/tmp_data_errlog_1'

def iter_data_preload(dataset, i):
    dataset.get_random_list()
    try:
        start_time = time.time()
        for count, (data, label) in enumerate(dataset):
            with open(config['out_dir']+str(i)+str('_')+str(count), 'wb') as handle:
                pickle.dump((data.astype(np.float16), label.astype(np.int16)), handle)
#             print(time.time()-start_time)
            with open(config['err_log'], 'a') as f:
                f.write(str(time.time()-start_time)+'\n')
            start_time = time.time()
            if ((count+1)%10000) == 0:
                print((count+1)//10000)
    except Exception:
        with open(config['err_log'], 'a') as f:
            traceback.print_exc(file=f)
        raise Exception

dataset = VoxIterableDataset(data_dir_dict, data_len_dict, config)
# dataset.noise_data_preload2mem()
dataset.noise_data_preload()

processes = [Process(target = iter_data_preload, args = (dataset, i)) for i in range(2)]
start_time = time.time()
[p.start() for p in processes]
joined = [p.join() for p in processes]
print(time.time()-start_time)

preloading music_dict
100
200
300
400
500
600
preloading noise_dict
100
200
300
400
500
600
700
800
900
preloading babble_dict
100
200
300
400
168.62345337867737


# Make train index

In [27]:
50*200

10000

In [28]:
# OPT_INDEX = '/Lun0/zhiyong/dataset'
train_data_dir = os.path.join(OPT_INDEX, 'train_data_small_50')
expected_len = 10000
workers = 2
single_worker_len = int(expected_len / workers)
output = os.path.join(OPT_INDEX, 'train_data_small_50.csv')

In [29]:
assert expected_len == len(glob.glob(train_data_dir+'/*'))

In [30]:
count = 0
with open(output, 'w') as f:
    for i in range(workers):
        for j in range(single_worker_len):
            path = os.path.join(train_data_dir, str(i)+'_'+str(j))
            assert os.path.isfile(path)
            f.write(path+'\n')
            count += 1

In [31]:
count

10000

# Make val data

In [45]:
OPT_INDEX = '/Lun0/zhiyong/sdsvc_t2_small_sparse'
OPT_INDEX_1 = '/Lun0/zhiyong/dataset'
OPT_INDEX_2 = '/Lun0/zhiyong/sdsvc_t2_train'
data_dir_dict = {}

# val
data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_val_dict_small_50')
data_dir_dict['music_dict'] = os.path.join(OPT_INDEX_1, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(OPT_INDEX_1, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(OPT_INDEX_1, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(OPT_INDEX_1, 'rir_dict')

In [46]:
data_len_dict = {}


data_len_dict['music_len'] = os.path.join(OPT_INDEX_1, 'music_len')
data_len_dict['noise_len'] = os.path.join(OPT_INDEX_1, 'noise_len')
data_len_dict['babble_len'] = os.path.join(OPT_INDEX_1, 'babble_len')

data_len_dict['spk2utt_train_len'] = os.path.join(OPT_INDEX_2, 'spk2utt_train_len')

In [47]:
config = {}

config['sr'] = 16000
config['repeats'] = 20
config['batch_size'] = 1
config['extended_prefectch'] = 1.0
config['out_dir'] = '/Lun0/zhiyong/sdsvc_t2_small_sparse/val_data_small_50/'

def iter_data_preload(dataset, i):
    dataset.get_random_list()
    start_time = time.time()
    for count, (data, label) in enumerate(dataset):
        with open(config['out_dir']+str(i)+'_'+str(count), 'wb') as handle:
            pickle.dump((data.astype(np.float16), label.astype(np.int16)), handle)
        print(time.time()-start_time)
        start_time = time.time()

dataset = VoxIterableDataset(data_dir_dict, data_len_dict, config)
dataset.noise_data_preload()

processes = [Process(target = iter_data_preload, args = (dataset, i)) for i in range(1)]
start_time = time.time()
[p.start() for p in processes]
joined = [p.join() for p in processes]
print(time.time()-start_time)

preloading music_dict
100
200
300
400
500
600
preloading noise_dict
100
200
300
400
500
600
700
800
900
preloading babble_dict
100
200
300
400
0.26771974563598633
0.05045938491821289
0.03043651580810547
0.036879777908325195
0.02369213104248047
0.06264352798461914
0.03606104850769043
0.019575119018554688
0.03874325752258301
0.022318363189697266
0.013471364974975586
0.012588739395141602
0.030272960662841797
0.03627181053161621
0.023572206497192383
0.030790328979492188
0.03085923194885254
0.023172616958618164
0.013617515563964844
0.013965129852294922
0.012416601181030273
0.026278018951416016
0.029240131378173828
0.03209352493286133
0.04804682731628418
0.02881765365600586
0.04365110397338867
0.0411989688873291
0.032058000564575195
0.01850152015686035
0.03261709213256836
0.01365351676940918
0.027460336685180664
0.03859519958496094
0.051972150802612305
0.026934385299682617
0.013690710067749023
0.018212080001831055
0.02089214324951172
0.026717662811279297
0.03316235542297363
0.018707513809204

0.0217893123626709
0.020052194595336914
0.0226747989654541
0.01520395278930664
0.012467145919799805
0.012309551239013672
0.028942584991455078
0.013670206069946289
0.014185190200805664
0.017148971557617188
0.028261184692382812
0.033797264099121094
0.012384414672851562
0.013563871383666992
0.015854835510253906
0.01360774040222168
0.012349367141723633
0.012291669845581055
0.01817035675048828
0.0236661434173584
0.015343904495239258
0.013727188110351562
0.013250589370727539
0.01668262481689453
0.017464637756347656
0.01365518569946289
0.013632059097290039
0.018909931182861328
0.02775406837463379
0.03725314140319824
0.02757406234741211
0.013758659362792969
0.01232290267944336
0.02746295928955078
0.01363062858581543
0.012401103973388672
0.012325048446655273
0.013588905334472656
0.023899078369140625
0.012368917465209961
0.026924848556518555
0.01898026466369629
0.017265796661376953
0.036031484603881836
0.014373779296875
0.03126120567321777
0.033109188079833984
0.013643264770507812
0.013601064682

0.01464533805847168
0.014644384384155273
0.034653425216674805
0.014687299728393555
0.023285388946533203
0.02029705047607422
0.014715909957885742
0.01788616180419922
0.023171424865722656
0.014731884002685547
0.02361583709716797
0.01475071907043457
0.02707195281982422
0.04051041603088379
0.022164344787597656
0.016163110733032227
0.023691415786743164
0.01894545555114746
0.03436994552612305
0.03296995162963867
0.03304910659790039
0.01801609992980957
0.030989408493041992
0.01472330093383789
0.015951156616210938
0.017048358917236328
0.015995264053344727
0.01590895652770996
0.0361020565032959
0.022417068481445312
0.01604938507080078
0.016085147857666016
0.018910646438598633
0.01605677604675293
0.015935182571411133
0.014688253402709961
0.02454543113708496
0.02064800262451172
0.015935182571411133
0.023714542388916016
0.01970505714416504
0.022141456604003906
0.015918731689453125
0.019176006317138672
0.01648879051208496
0.02060413360595703
0.03004908561706543
0.01774764060974121
0.028271198272705

# Make validation index (Need only once)

In [48]:
OPT_INDEX = '/Lun0/zhiyong/sdsvc_t2_small_sparse'
train_data_dir = os.path.join(OPT_INDEX, 'val_data_small_50')
expected_len = 50 * 20
workers = 1
single_worker_len = int(expected_len / workers)
output = os.path.join(OPT_INDEX, 'val_data_small_50.csv')

In [49]:
assert expected_len == len(glob.glob(train_data_dir+'/*'))

In [50]:
count = 0
with open(output, 'w') as f:
    for i in range(workers):
        for j in range(single_worker_len):
            path = os.path.join(train_data_dir, str(i)+'_'+str(j))
            assert os.path.isfile(path)
            f.write(path+'\n')
            count += 1

In [51]:
count

1000

# make small 150 train

In [52]:
OPT_INDEX = '/Lun0/zhiyong/sdsvc_t2_small_sparse'
OPT_INDEX_1 = '/Lun0/zhiyong/dataset'
OPT_INDEX_2 = '/Lun0/zhiyong/sdsvc_t2_train'
data_dir_dict = {}

# train
# data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_train_dict')
data_dir_dict['music_dict'] = os.path.join(OPT_INDEX_1, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(OPT_INDEX_1, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(OPT_INDEX_1, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(OPT_INDEX_1, 'rir_dict')

In [53]:
data_len_dict = {}

data_len_dict['music_len'] = os.path.join(OPT_INDEX_1, 'music_len')
data_len_dict['noise_len'] = os.path.join(OPT_INDEX_1, 'noise_len')
data_len_dict['babble_len'] = os.path.join(OPT_INDEX_1, 'babble_len')
data_len_dict['spk2utt_train_len'] = os.path.join(OPT_INDEX_2, 'spk2utt_train_len')
# data_len_dict['spk2utt_val_len'] = os.path.join(OPT_INDEX, 'spk2utt_val_len')

In [54]:
with open(os.path.join(OPT_INDEX, 'spk2utt_train_dict_small'), 'rb') as handle:
    spk2utt_train_dict = pickle.load(handle)

In [55]:
spk2utt_train_dict.keys()

dict_keys([1211, 1212, 1213, 1214, 1215, 1216, 1217, 1218, 1219, 1220, 1221, 1222, 1223, 1224, 1225, 1226, 1227, 1228, 1229, 1230, 1231, 1232, 1233, 1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241, 1242, 1243, 1244, 1245, 1246, 1247, 1248, 1249, 1250, 1251, 1252, 1253, 1254, 1255, 1256, 1257, 1258, 1259, 1260, 1261, 1262, 1263, 1264, 1265, 1266, 1267, 1268, 1269, 1270, 1271, 1272, 1273, 1274, 1275, 1276, 1277, 1278, 1279, 1280, 1281, 1282, 1283, 1284, 1285, 1286, 1287, 1288, 1289, 1290, 1291, 1292, 1293, 1294, 1295, 1296, 1297, 1298, 1299, 1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309, 1310, 1311, 1312, 1313, 1314, 1315, 1316, 1317, 1318, 1319, 1320, 1321, 1322, 1323, 1324, 1325, 1326, 1327, 1328, 1329, 1330, 1331, 1332, 1333, 1334, 1335, 1336, 1337, 1338, 1339, 1340, 1341, 1342, 1343, 1344, 1345, 1346, 1347, 1348, 1349, 1350, 1351, 1352, 1353, 1354, 1355, 1356, 1357, 1358, 1359, 1360, 1361, 1362, 1363, 1364, 1365, 1366, 1367, 1368, 1369, 1370, 1371, 1372, 1373, 1374, 1375,

In [56]:
spk2utt_train_dict_small_150 = {}
spk2utt_val_dict_small_150 = {}
for count, i in enumerate(spk2utt_train_dict):
    if count < 150:
        spk2utt_train_dict_small_150[1211+count] = spk2utt_train_dict[i]
    else:
        break

In [57]:
spk2utt_train_dict_small_150.keys()

dict_keys([1211, 1212, 1213, 1214, 1215, 1216, 1217, 1218, 1219, 1220, 1221, 1222, 1223, 1224, 1225, 1226, 1227, 1228, 1229, 1230, 1231, 1232, 1233, 1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241, 1242, 1243, 1244, 1245, 1246, 1247, 1248, 1249, 1250, 1251, 1252, 1253, 1254, 1255, 1256, 1257, 1258, 1259, 1260, 1261, 1262, 1263, 1264, 1265, 1266, 1267, 1268, 1269, 1270, 1271, 1272, 1273, 1274, 1275, 1276, 1277, 1278, 1279, 1280, 1281, 1282, 1283, 1284, 1285, 1286, 1287, 1288, 1289, 1290, 1291, 1292, 1293, 1294, 1295, 1296, 1297, 1298, 1299, 1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309, 1310, 1311, 1312, 1313, 1314, 1315, 1316, 1317, 1318, 1319, 1320, 1321, 1322, 1323, 1324, 1325, 1326, 1327, 1328, 1329, 1330, 1331, 1332, 1333, 1334, 1335, 1336, 1337, 1338, 1339, 1340, 1341, 1342, 1343, 1344, 1345, 1346, 1347, 1348, 1349, 1350, 1351, 1352, 1353, 1354, 1355, 1356, 1357, 1358, 1359, 1360])

In [58]:
spk2utt_train_dict_small_150[1211]

['/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101064.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101065.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101066.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101067.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101068.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101069.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101070.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101071.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101072.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101073.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101074.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101075.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101076.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/tas

In [59]:
for i in spk2utt_train_dict_small_150:
    spk2utt_val_dict_small_150[i] = [spk2utt_train_dict_small_150[i][0]]
    spk2utt_train_dict_small_150[i].pop(0)

In [60]:
spk2utt_train_dict_small_150[1211]

['/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101065.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101066.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101067.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101068.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101069.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101070.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101071.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101072.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101073.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101074.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101075.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101076.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101077.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/tas

In [61]:
spk2utt_val_dict_small_150[1211]

['/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101064.wav']

In [62]:
with open(os.path.join(OPT_INDEX, 'spk2utt_train_dict_small_150'), 'wb') as handle:
    pickle.dump(spk2utt_train_dict_small_150, handle)
with open(os.path.join(OPT_INDEX, 'spk2utt_val_dict_small_150'), 'wb') as handle:
    pickle.dump(spk2utt_val_dict_small_150, handle)

In [63]:
data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_train_dict_small_150')

In [64]:
150 * 200

30000

In [65]:
config = {}

config['sr'] = 16000
config['repeats'] = 100
config['batch_size'] = 1
config['extended_prefectch'] = 1.0
config['out_dir'] = '/Lun0/zhiyong/sdsvc_t2_small_sparse/train_data_small_150/'
config['err_log'] = '/Lun0/zhiyong/sdsvc_t2_small_sparse/tmp_data_errlog_1'

def iter_data_preload(dataset, i):
    dataset.get_random_list()
    try:
        start_time = time.time()
        for count, (data, label) in enumerate(dataset):
            with open(config['out_dir']+str(i)+str('_')+str(count), 'wb') as handle:
                pickle.dump((data.astype(np.float16), label.astype(np.int16)), handle)
#             print(time.time()-start_time)
            with open(config['err_log'], 'a') as f:
                f.write(str(time.time()-start_time)+'\n')
            start_time = time.time()
            if ((count+1)%10000) == 0:
                print((count+1)//10000)
    except Exception:
        with open(config['err_log'], 'a') as f:
            traceback.print_exc(file=f)
        raise Exception

dataset = VoxIterableDataset(data_dir_dict, data_len_dict, config)
# dataset.noise_data_preload2mem()
dataset.noise_data_preload()

processes = [Process(target = iter_data_preload, args = (dataset, i)) for i in range(2)]
start_time = time.time()
[p.start() for p in processes]
joined = [p.join() for p in processes]
print(time.time()-start_time)

preloading music_dict
100
200
300
400
500
600
preloading noise_dict
100
200
300
400
500
600
700
800
900
preloading babble_dict
100
200
300
400
1
1
483.8709132671356


# Make train index

In [66]:
150*200

30000

In [67]:
# OPT_INDEX = '/Lun0/zhiyong/dataset'
train_data_dir = os.path.join(OPT_INDEX, 'train_data_small_150')
expected_len = 30000
workers = 2
single_worker_len = int(expected_len / workers)
output = os.path.join(OPT_INDEX, 'train_data_small_150.csv')

In [68]:
assert expected_len == len(glob.glob(train_data_dir+'/*'))

In [69]:
count = 0
with open(output, 'w') as f:
    for i in range(workers):
        for j in range(single_worker_len):
            path = os.path.join(train_data_dir, str(i)+'_'+str(j))
            assert os.path.isfile(path)
            f.write(path+'\n')
            count += 1

In [70]:
count

30000

# Make val data

In [71]:
OPT_INDEX = '/Lun0/zhiyong/sdsvc_t2_small_sparse'
OPT_INDEX_1 = '/Lun0/zhiyong/dataset'
OPT_INDEX_2 = '/Lun0/zhiyong/sdsvc_t2_train'
data_dir_dict = {}

# val
data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_val_dict_small_150')
data_dir_dict['music_dict'] = os.path.join(OPT_INDEX_1, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(OPT_INDEX_1, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(OPT_INDEX_1, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(OPT_INDEX_1, 'rir_dict')

In [72]:
data_len_dict = {}


data_len_dict['music_len'] = os.path.join(OPT_INDEX_1, 'music_len')
data_len_dict['noise_len'] = os.path.join(OPT_INDEX_1, 'noise_len')
data_len_dict['babble_len'] = os.path.join(OPT_INDEX_1, 'babble_len')

data_len_dict['spk2utt_train_len'] = os.path.join(OPT_INDEX_2, 'spk2utt_train_len')

In [73]:
config = {}

config['sr'] = 16000
config['repeats'] = 10
config['batch_size'] = 1
config['extended_prefectch'] = 1.0
config['out_dir'] = '/Lun0/zhiyong/sdsvc_t2_small_sparse/val_data_small_150/'

def iter_data_preload(dataset, i):
    dataset.get_random_list()
    start_time = time.time()
    for count, (data, label) in enumerate(dataset):
        with open(config['out_dir']+str(i)+'_'+str(count), 'wb') as handle:
            pickle.dump((data.astype(np.float16), label.astype(np.int16)), handle)
        print(time.time()-start_time)
        start_time = time.time()

dataset = VoxIterableDataset(data_dir_dict, data_len_dict, config)
dataset.noise_data_preload()

processes = [Process(target = iter_data_preload, args = (dataset, i)) for i in range(1)]
start_time = time.time()
[p.start() for p in processes]
joined = [p.join() for p in processes]
print(time.time()-start_time)

preloading music_dict
100
200
300
400
500
600
preloading noise_dict
100
200
300
400
500
600
700
800
900
preloading babble_dict
100
200
300
400
0.03435921669006348
0.03234601020812988
0.044571638107299805
0.022350072860717773
0.03440499305725098
0.01973271369934082
0.02081608772277832
0.019475460052490234
0.009073019027709961
0.019533872604370117
0.03326106071472168
0.025654077529907227
0.026387929916381836
0.00927734375
0.02250051498413086
0.009009838104248047
0.01989269256591797
0.020903825759887695
0.019164085388183594
0.013195991516113281
0.02103877067565918
0.007967948913574219
0.01782536506652832
0.012341022491455078
0.03480648994445801
0.007882118225097656
0.007739067077636719
0.02938103675842285
0.01752185821533203
0.007786273956298828
0.008832216262817383
0.008113384246826172
0.008872032165527344
0.01717662811279297
0.012746810913085938
0.016577482223510742
0.026482820510864258
0.016200542449951172
0.030149221420288086
0.03929901123046875
0.016392946243286133
0.0079207420349121

0.020160436630249023
0.01377558708190918
0.012524604797363281
0.020815610885620117
0.017252206802368164
0.017233610153198242
0.017290830612182617
0.01537013053894043
0.013822078704833984
0.013669967651367188
0.02849292755126953
0.013667821884155273
0.013685226440429688
0.01986837387084961
0.013624429702758789
0.016329050064086914
0.019115924835205078
0.022598981857299805
0.013733625411987305
0.01251673698425293
0.01535797119140625
0.012529373168945312
0.017259597778320312
0.017194747924804688
0.02602672576904297
0.027999401092529297
0.015408992767333984
0.03538036346435547
0.028619766235351562
0.027894258499145508
0.014842033386230469
0.025177955627441406
0.012720584869384766
0.02607250213623047
0.01640939712524414
0.013664484024047852
0.026979446411132812
0.0338437557220459
0.027192115783691406
0.014205455780029297
0.013805150985717773
0.02415013313293457
0.017258405685424805
0.01383662223815918
0.013833999633789062
0.020220279693603516
0.01240229606628418
0.020287275314331055
0.02935

0.018181800842285156
0.012394428253173828
0.01362466812133789
0.013735294342041016
0.012414693832397461
0.018465757369995117
0.015595197677612305
0.022382736206054688
0.013697385787963867
0.013724803924560547
0.01548147201538086
0.015514373779296875
0.01776289939880371
0.013762474060058594
0.012467145919799805
0.02996230125427246
0.012511491775512695
0.013757467269897461
0.029703140258789062
0.02634716033935547
0.013662576675415039
0.022443532943725586
0.015127897262573242
0.01722431182861328
0.020196199417114258
0.015563488006591797
0.029803991317749023
0.015646696090698242
0.012963056564331055
0.008838176727294922
0.008831262588500977
0.01380157470703125
0.008938789367675781
0.012207269668579102
0.010202884674072266
0.007864952087402344
0.011579036712646484
0.026619672775268555
0.01024317741394043
0.013258695602416992
0.008803367614746094
0.009354829788208008
0.008812904357910156
0.00882577896118164
0.012274980545043945
0.01544332504272461
0.012272119522094727
0.016238927841186523
0.

0.008814573287963867
0.008799314498901367
0.015459775924682617
0.008101224899291992
0.020203351974487305
0.010205984115600586
0.007880210876464844
0.007790088653564453
0.007831335067749023
0.008816003799438477
0.007833003997802734
0.01803898811340332
0.036351919174194336
0.017111778259277344
0.01970219612121582
0.015958309173583984
0.01734018325805664
0.03644251823425293
0.013884305953979492
0.013742923736572266
0.013666629791259766
0.015514135360717773
0.012433767318725586
0.016814470291137695
0.03248953819274902
0.018555879592895508
0.012539386749267578
0.017437458038330078
0.01268148422241211
0.014452219009399414
0.012503862380981445
0.01371455192565918
0.012410402297973633
0.015190839767456055
0.03719687461853027
0.024264097213745117
0.015406370162963867
0.03392910957336426
0.01773858070373535
0.01548314094543457
0.015277385711669922
0.013732433319091797
0.013677358627319336
0.03355741500854492
0.02826094627380371
0.012414932250976562
0.020965576171875
0.01381683349609375
0.0223526

# Make validation index (Need only once)

In [74]:
OPT_INDEX = '/Lun0/zhiyong/sdsvc_t2_small_sparse'
train_data_dir = os.path.join(OPT_INDEX, 'val_data_small_150')
expected_len = 150 * 10
workers = 1
single_worker_len = int(expected_len / workers)
output = os.path.join(OPT_INDEX, 'val_data_small_150.csv')

In [75]:
assert expected_len == len(glob.glob(train_data_dir+'/*'))

In [76]:
count = 0
with open(output, 'w') as f:
    for i in range(workers):
        for j in range(single_worker_len):
            path = os.path.join(train_data_dir, str(i)+'_'+str(j))
            assert os.path.isfile(path)
            f.write(path+'\n')
            count += 1

In [77]:
count

1500

# make small 200 train

In [78]:
OPT_INDEX = '/Lun0/zhiyong/sdsvc_t2_small_sparse'
OPT_INDEX_1 = '/Lun0/zhiyong/dataset'
OPT_INDEX_2 = '/Lun0/zhiyong/sdsvc_t2_train'
data_dir_dict = {}

# train
# data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_train_dict')
data_dir_dict['music_dict'] = os.path.join(OPT_INDEX_1, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(OPT_INDEX_1, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(OPT_INDEX_1, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(OPT_INDEX_1, 'rir_dict')

In [79]:
data_len_dict = {}

data_len_dict['music_len'] = os.path.join(OPT_INDEX_1, 'music_len')
data_len_dict['noise_len'] = os.path.join(OPT_INDEX_1, 'noise_len')
data_len_dict['babble_len'] = os.path.join(OPT_INDEX_1, 'babble_len')
data_len_dict['spk2utt_train_len'] = os.path.join(OPT_INDEX_2, 'spk2utt_train_len')
# data_len_dict['spk2utt_val_len'] = os.path.join(OPT_INDEX, 'spk2utt_val_len')

In [80]:
with open(os.path.join(OPT_INDEX, 'spk2utt_train_dict_small'), 'rb') as handle:
    spk2utt_train_dict = pickle.load(handle)

In [81]:
spk2utt_train_dict.keys()

dict_keys([1211, 1212, 1213, 1214, 1215, 1216, 1217, 1218, 1219, 1220, 1221, 1222, 1223, 1224, 1225, 1226, 1227, 1228, 1229, 1230, 1231, 1232, 1233, 1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241, 1242, 1243, 1244, 1245, 1246, 1247, 1248, 1249, 1250, 1251, 1252, 1253, 1254, 1255, 1256, 1257, 1258, 1259, 1260, 1261, 1262, 1263, 1264, 1265, 1266, 1267, 1268, 1269, 1270, 1271, 1272, 1273, 1274, 1275, 1276, 1277, 1278, 1279, 1280, 1281, 1282, 1283, 1284, 1285, 1286, 1287, 1288, 1289, 1290, 1291, 1292, 1293, 1294, 1295, 1296, 1297, 1298, 1299, 1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309, 1310, 1311, 1312, 1313, 1314, 1315, 1316, 1317, 1318, 1319, 1320, 1321, 1322, 1323, 1324, 1325, 1326, 1327, 1328, 1329, 1330, 1331, 1332, 1333, 1334, 1335, 1336, 1337, 1338, 1339, 1340, 1341, 1342, 1343, 1344, 1345, 1346, 1347, 1348, 1349, 1350, 1351, 1352, 1353, 1354, 1355, 1356, 1357, 1358, 1359, 1360, 1361, 1362, 1363, 1364, 1365, 1366, 1367, 1368, 1369, 1370, 1371, 1372, 1373, 1374, 1375,

In [82]:
spk2utt_train_dict_small_200 = {}
spk2utt_val_dict_small_200 = {}
for count, i in enumerate(spk2utt_train_dict):
    if count < 200:
        spk2utt_train_dict_small_200[1211+count] = spk2utt_train_dict[i]
    else:
        break

In [83]:
spk2utt_train_dict_small_200.keys()

dict_keys([1211, 1212, 1213, 1214, 1215, 1216, 1217, 1218, 1219, 1220, 1221, 1222, 1223, 1224, 1225, 1226, 1227, 1228, 1229, 1230, 1231, 1232, 1233, 1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241, 1242, 1243, 1244, 1245, 1246, 1247, 1248, 1249, 1250, 1251, 1252, 1253, 1254, 1255, 1256, 1257, 1258, 1259, 1260, 1261, 1262, 1263, 1264, 1265, 1266, 1267, 1268, 1269, 1270, 1271, 1272, 1273, 1274, 1275, 1276, 1277, 1278, 1279, 1280, 1281, 1282, 1283, 1284, 1285, 1286, 1287, 1288, 1289, 1290, 1291, 1292, 1293, 1294, 1295, 1296, 1297, 1298, 1299, 1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309, 1310, 1311, 1312, 1313, 1314, 1315, 1316, 1317, 1318, 1319, 1320, 1321, 1322, 1323, 1324, 1325, 1326, 1327, 1328, 1329, 1330, 1331, 1332, 1333, 1334, 1335, 1336, 1337, 1338, 1339, 1340, 1341, 1342, 1343, 1344, 1345, 1346, 1347, 1348, 1349, 1350, 1351, 1352, 1353, 1354, 1355, 1356, 1357, 1358, 1359, 1360, 1361, 1362, 1363, 1364, 1365, 1366, 1367, 1368, 1369, 1370, 1371, 1372, 1373, 1374, 1375,

In [84]:
spk2utt_train_dict_small_200[1211]

['/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101064.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101065.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101066.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101067.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101068.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101069.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101070.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101071.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101072.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101073.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101074.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101075.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101076.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/tas

In [85]:
for i in spk2utt_train_dict_small_200:
    spk2utt_val_dict_small_200[i] = [spk2utt_train_dict_small_200[i][0]]
    spk2utt_train_dict_small_200[i].pop(0)

In [86]:
spk2utt_train_dict_small_200[1211]

['/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101065.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101066.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101067.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101068.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101069.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101070.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101071.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101072.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101073.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101074.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101075.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101076.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101077.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/tas

In [87]:
spk2utt_val_dict_small_200[1211]

['/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101064.wav']

In [88]:
with open(os.path.join(OPT_INDEX, 'spk2utt_train_dict_small_200'), 'wb') as handle:
    pickle.dump(spk2utt_train_dict_small_200, handle)
with open(os.path.join(OPT_INDEX, 'spk2utt_val_dict_small_200'), 'wb') as handle:
    pickle.dump(spk2utt_val_dict_small_200, handle)

In [89]:
data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_train_dict_small_200')

In [90]:
200 * 200

40000

In [91]:
config = {}

config['sr'] = 16000
config['repeats'] = 100
config['batch_size'] = 1
config['extended_prefectch'] = 1.0
config['out_dir'] = '/Lun0/zhiyong/sdsvc_t2_small_sparse/train_data_small_200/'
config['err_log'] = '/Lun0/zhiyong/sdsvc_t2_small_sparse/tmp_data_errlog_1'

def iter_data_preload(dataset, i):
    dataset.get_random_list()
    try:
        start_time = time.time()
        for count, (data, label) in enumerate(dataset):
            with open(config['out_dir']+str(i)+str('_')+str(count), 'wb') as handle:
                pickle.dump((data.astype(np.float16), label.astype(np.int16)), handle)
#             print(time.time()-start_time)
            with open(config['err_log'], 'a') as f:
                f.write(str(time.time()-start_time)+'\n')
            start_time = time.time()
            if ((count+1)%10000) == 0:
                print((count+1)//10000)
    except Exception:
        with open(config['err_log'], 'a') as f:
            traceback.print_exc(file=f)
        raise Exception

dataset = VoxIterableDataset(data_dir_dict, data_len_dict, config)
# dataset.noise_data_preload2mem()
dataset.noise_data_preload()

processes = [Process(target = iter_data_preload, args = (dataset, i)) for i in range(2)]
start_time = time.time()
[p.start() for p in processes]
joined = [p.join() for p in processes]
print(time.time()-start_time)

preloading music_dict
100
200
300
400
500
600
preloading noise_dict
100
200
300
400
500
600
700
800
900
preloading babble_dict
100
200
300
400
1
1
2
2
525.0212845802307


# Make train index

In [92]:
200*200

40000

In [93]:
# OPT_INDEX = '/Lun0/zhiyong/dataset'
train_data_dir = os.path.join(OPT_INDEX, 'train_data_small_200')
expected_len = 40000
workers = 2
single_worker_len = int(expected_len / workers)
output = os.path.join(OPT_INDEX, 'train_data_small_200.csv')

In [94]:
assert expected_len == len(glob.glob(train_data_dir+'/*'))

In [95]:
count = 0
with open(output, 'w') as f:
    for i in range(workers):
        for j in range(single_worker_len):
            path = os.path.join(train_data_dir, str(i)+'_'+str(j))
            assert os.path.isfile(path)
            f.write(path+'\n')
            count += 1

In [96]:
count

40000

# Make val data

In [97]:
OPT_INDEX = '/Lun0/zhiyong/sdsvc_t2_small_sparse'
OPT_INDEX_1 = '/Lun0/zhiyong/dataset'
OPT_INDEX_2 = '/Lun0/zhiyong/sdsvc_t2_train'
data_dir_dict = {}

# val
data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_val_dict_small_200')
data_dir_dict['music_dict'] = os.path.join(OPT_INDEX_1, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(OPT_INDEX_1, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(OPT_INDEX_1, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(OPT_INDEX_1, 'rir_dict')

In [98]:
data_len_dict = {}


data_len_dict['music_len'] = os.path.join(OPT_INDEX_1, 'music_len')
data_len_dict['noise_len'] = os.path.join(OPT_INDEX_1, 'noise_len')
data_len_dict['babble_len'] = os.path.join(OPT_INDEX_1, 'babble_len')

data_len_dict['spk2utt_train_len'] = os.path.join(OPT_INDEX_2, 'spk2utt_train_len')

In [99]:
config = {}

config['sr'] = 16000
config['repeats'] = 5
config['batch_size'] = 1
config['extended_prefectch'] = 1.0
config['out_dir'] = '/Lun0/zhiyong/sdsvc_t2_small_sparse/val_data_small_200/'

def iter_data_preload(dataset, i):
    dataset.get_random_list()
    start_time = time.time()
    for count, (data, label) in enumerate(dataset):
        with open(config['out_dir']+str(i)+'_'+str(count), 'wb') as handle:
            pickle.dump((data.astype(np.float16), label.astype(np.int16)), handle)
        print(time.time()-start_time)
        start_time = time.time()

dataset = VoxIterableDataset(data_dir_dict, data_len_dict, config)
dataset.noise_data_preload()

processes = [Process(target = iter_data_preload, args = (dataset, i)) for i in range(1)]
start_time = time.time()
[p.start() for p in processes]
joined = [p.join() for p in processes]
print(time.time()-start_time)

preloading music_dict
100
200
300
400
500
600
preloading noise_dict
100
200
300
400
500
600
700
800
900
preloading babble_dict
100
200
300
400
0.030176162719726562
0.014820575714111328
0.009829044342041016
0.018523454666137695
0.01649928092956543
0.01048588752746582
0.02793407440185547
0.01302790641784668
0.020483970642089844
0.01832413673400879
0.012751102447509766
0.012466192245483398
0.019525527954101562
0.00877237319946289
0.01599717140197754
0.011669635772705078
0.01488351821899414
0.008764266967773438
0.015045642852783203
0.02087855339050293
0.007860898971557617
0.015416622161865234
0.009523153305053711
0.02212977409362793
0.008767843246459961
0.013015031814575195
0.00883340835571289
0.008719682693481445
0.008759021759033203
0.01037740707397461
0.012049674987792969
0.010308504104614258
0.007084369659423828
0.007693290710449219
0.012017488479614258
0.010993003845214844
0.012515068054199219
0.018311023712158203
0.013835668563842773
0.008620262145996094
0.007695674896240234
0.008671

0.024495363235473633
0.010222434997558594
0.012837409973144531
0.013819456100463867
0.007823467254638672
0.012671232223510742
0.008842229843139648
0.017775297164916992
0.010489463806152344
0.018479347229003906
0.016951322555541992
0.013675928115844727
0.012436866760253906
0.014653444290161133
0.013162374496459961
0.008912086486816406
0.014635324478149414
0.008867740631103516
0.016129016876220703
0.007795810699462891
0.016173601150512695
0.015331268310546875
0.009392738342285156
0.008874893188476562
0.0127410888671875
0.008936405181884766
0.009647607803344727
0.014582395553588867
0.011549949645996094
0.008832454681396484
0.021133899688720703
0.013229131698608398
0.02629399299621582
0.02135777473449707
0.008860111236572266
0.009241819381713867
0.014049291610717773
0.023529052734375
0.016195058822631836
0.00877070426940918
0.007792472839355469
0.015461444854736328
0.02312183380126953
0.007841348648071289
0.008754968643188477
0.021192312240600586
0.007815361022949219
0.00925588607788086
0.

0.011392831802368164
0.01201176643371582
0.014271259307861328
0.008312702178955078
0.013563871383666992
0.009322166442871094
0.008323907852172852
0.016345977783203125
0.028180599212646484
0.012590169906616211
0.01975250244140625
0.03370022773742676
0.01940608024597168
0.017077922821044922
0.010754108428955078
0.01115560531616211
0.009438276290893555
0.010741710662841797
0.00838613510131836
0.012876510620117188
0.009382009506225586
0.019437313079833984
0.023731470108032227
0.008320331573486328
0.014571905136108398
0.008320093154907227
0.014878034591674805
0.009400129318237305
0.017461061477661133
0.008287429809570312
0.013617753982543945
0.008289575576782227
0.021871328353881836
0.008524179458618164
0.009355545043945312
0.009781837463378906
0.017263412475585938
0.008287429809570312
0.013652563095092773
0.0083160400390625
0.018799304962158203
0.010800361633300781
0.013517379760742188
0.008336305618286133
0.014554738998413086
0.011835813522338867
0.009952783584594727
0.008236885070800781


# Make validation index (Need only once)

In [100]:
OPT_INDEX = '/Lun0/zhiyong/sdsvc_t2_small_sparse'
train_data_dir = os.path.join(OPT_INDEX, 'val_data_small_200')
expected_len = 200 * 5
workers = 1
single_worker_len = int(expected_len / workers)
output = os.path.join(OPT_INDEX, 'val_data_small_200.csv')

In [101]:
assert expected_len == len(glob.glob(train_data_dir+'/*'))

In [102]:
count = 0
with open(output, 'w') as f:
    for i in range(workers):
        for j in range(single_worker_len):
            path = os.path.join(train_data_dir, str(i)+'_'+str(j))
            assert os.path.isfile(path)
            f.write(path+'\n')
            count += 1

In [103]:
count

1000

# make small 300 train

In [104]:
OPT_INDEX = '/Lun0/zhiyong/sdsvc_t2_small_sparse'
OPT_INDEX_1 = '/Lun0/zhiyong/dataset'
OPT_INDEX_2 = '/Lun0/zhiyong/sdsvc_t2_train'
data_dir_dict = {}

# train
# data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_train_dict')
data_dir_dict['music_dict'] = os.path.join(OPT_INDEX_1, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(OPT_INDEX_1, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(OPT_INDEX_1, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(OPT_INDEX_1, 'rir_dict')

In [105]:
data_len_dict = {}

data_len_dict['music_len'] = os.path.join(OPT_INDEX_1, 'music_len')
data_len_dict['noise_len'] = os.path.join(OPT_INDEX_1, 'noise_len')
data_len_dict['babble_len'] = os.path.join(OPT_INDEX_1, 'babble_len')
data_len_dict['spk2utt_train_len'] = os.path.join(OPT_INDEX_2, 'spk2utt_train_len')
# data_len_dict['spk2utt_val_len'] = os.path.join(OPT_INDEX, 'spk2utt_val_len')

In [106]:
with open(os.path.join(OPT_INDEX, 'spk2utt_train_dict_small'), 'rb') as handle:
    spk2utt_train_dict = pickle.load(handle)

In [107]:
spk2utt_train_dict.keys()

dict_keys([1211, 1212, 1213, 1214, 1215, 1216, 1217, 1218, 1219, 1220, 1221, 1222, 1223, 1224, 1225, 1226, 1227, 1228, 1229, 1230, 1231, 1232, 1233, 1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241, 1242, 1243, 1244, 1245, 1246, 1247, 1248, 1249, 1250, 1251, 1252, 1253, 1254, 1255, 1256, 1257, 1258, 1259, 1260, 1261, 1262, 1263, 1264, 1265, 1266, 1267, 1268, 1269, 1270, 1271, 1272, 1273, 1274, 1275, 1276, 1277, 1278, 1279, 1280, 1281, 1282, 1283, 1284, 1285, 1286, 1287, 1288, 1289, 1290, 1291, 1292, 1293, 1294, 1295, 1296, 1297, 1298, 1299, 1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309, 1310, 1311, 1312, 1313, 1314, 1315, 1316, 1317, 1318, 1319, 1320, 1321, 1322, 1323, 1324, 1325, 1326, 1327, 1328, 1329, 1330, 1331, 1332, 1333, 1334, 1335, 1336, 1337, 1338, 1339, 1340, 1341, 1342, 1343, 1344, 1345, 1346, 1347, 1348, 1349, 1350, 1351, 1352, 1353, 1354, 1355, 1356, 1357, 1358, 1359, 1360, 1361, 1362, 1363, 1364, 1365, 1366, 1367, 1368, 1369, 1370, 1371, 1372, 1373, 1374, 1375,

In [108]:
spk2utt_train_dict_small_300 = {}
spk2utt_val_dict_small_300 = {}
for count, i in enumerate(spk2utt_train_dict):
    if count < 300:
        spk2utt_train_dict_small_300[1211+count] = spk2utt_train_dict[i]
    else:
        break

In [109]:
spk2utt_train_dict_small_300.keys()

dict_keys([1211, 1212, 1213, 1214, 1215, 1216, 1217, 1218, 1219, 1220, 1221, 1222, 1223, 1224, 1225, 1226, 1227, 1228, 1229, 1230, 1231, 1232, 1233, 1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241, 1242, 1243, 1244, 1245, 1246, 1247, 1248, 1249, 1250, 1251, 1252, 1253, 1254, 1255, 1256, 1257, 1258, 1259, 1260, 1261, 1262, 1263, 1264, 1265, 1266, 1267, 1268, 1269, 1270, 1271, 1272, 1273, 1274, 1275, 1276, 1277, 1278, 1279, 1280, 1281, 1282, 1283, 1284, 1285, 1286, 1287, 1288, 1289, 1290, 1291, 1292, 1293, 1294, 1295, 1296, 1297, 1298, 1299, 1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309, 1310, 1311, 1312, 1313, 1314, 1315, 1316, 1317, 1318, 1319, 1320, 1321, 1322, 1323, 1324, 1325, 1326, 1327, 1328, 1329, 1330, 1331, 1332, 1333, 1334, 1335, 1336, 1337, 1338, 1339, 1340, 1341, 1342, 1343, 1344, 1345, 1346, 1347, 1348, 1349, 1350, 1351, 1352, 1353, 1354, 1355, 1356, 1357, 1358, 1359, 1360, 1361, 1362, 1363, 1364, 1365, 1366, 1367, 1368, 1369, 1370, 1371, 1372, 1373, 1374, 1375,

In [110]:
spk2utt_train_dict_small_300[1211]

['/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101064.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101065.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101066.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101067.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101068.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101069.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101070.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101071.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101072.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101073.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101074.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101075.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101076.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/tas

In [111]:
for i in spk2utt_train_dict_small_300:
    spk2utt_val_dict_small_300[i] = [spk2utt_train_dict_small_300[i][0]]
    spk2utt_train_dict_small_300[i].pop(0)

In [112]:
spk2utt_train_dict_small_300[1211]

['/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101065.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101066.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101067.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101068.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101069.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101070.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101071.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101072.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101073.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101074.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101075.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101076.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101077.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/tas

In [113]:
spk2utt_val_dict_small_300[1211]

['/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101064.wav']

In [114]:
with open(os.path.join(OPT_INDEX, 'spk2utt_train_dict_small_300'), 'wb') as handle:
    pickle.dump(spk2utt_train_dict_small_300, handle)
with open(os.path.join(OPT_INDEX, 'spk2utt_val_dict_small_300'), 'wb') as handle:
    pickle.dump(spk2utt_val_dict_small_300, handle)

In [115]:
data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_train_dict_small_300')

In [116]:
300 * 200

60000

In [117]:
config = {}

config['sr'] = 16000
config['repeats'] = 100
config['batch_size'] = 1
config['extended_prefectch'] = 1.0
config['out_dir'] = '/Lun0/zhiyong/sdsvc_t2_small_sparse/train_data_small_300/'
config['err_log'] = '/Lun0/zhiyong/sdsvc_t2_small_sparse/tmp_data_errlog_1'

def iter_data_preload(dataset, i):
    dataset.get_random_list()
    try:
        start_time = time.time()
        for count, (data, label) in enumerate(dataset):
            with open(config['out_dir']+str(i)+str('_')+str(count), 'wb') as handle:
                pickle.dump((data.astype(np.float16), label.astype(np.int16)), handle)
#             print(time.time()-start_time)
            with open(config['err_log'], 'a') as f:
                f.write(str(time.time()-start_time)+'\n')
            start_time = time.time()
            if ((count+1)%10000) == 0:
                print((count+1)//10000)
    except Exception:
        with open(config['err_log'], 'a') as f:
            traceback.print_exc(file=f)
        raise Exception

dataset = VoxIterableDataset(data_dir_dict, data_len_dict, config)
# dataset.noise_data_preload2mem()
dataset.noise_data_preload()

processes = [Process(target = iter_data_preload, args = (dataset, i)) for i in range(2)]
start_time = time.time()
[p.start() for p in processes]
joined = [p.join() for p in processes]
print(time.time()-start_time)

preloading music_dict
100
200
300
400
500
600
preloading noise_dict
100
200
300
400
500
600
700
800
900
preloading babble_dict
100
200
300
400
1
1
2
2
3
3
862.6489877700806


# Make train index

In [118]:
300*200

60000

In [119]:
# OPT_INDEX = '/Lun0/zhiyong/dataset'
train_data_dir = os.path.join(OPT_INDEX, 'train_data_small_300')
expected_len = 60000
workers = 2
single_worker_len = int(expected_len / workers)
output = os.path.join(OPT_INDEX, 'train_data_small_300.csv')

In [120]:
assert expected_len == len(glob.glob(train_data_dir+'/*'))

In [121]:
count = 0
with open(output, 'w') as f:
    for i in range(workers):
        for j in range(single_worker_len):
            path = os.path.join(train_data_dir, str(i)+'_'+str(j))
            assert os.path.isfile(path)
            f.write(path+'\n')
            count += 1

In [122]:
count

60000

# Make val data

In [123]:
OPT_INDEX = '/Lun0/zhiyong/sdsvc_t2_small_sparse'
OPT_INDEX_1 = '/Lun0/zhiyong/dataset'
OPT_INDEX_2 = '/Lun0/zhiyong/sdsvc_t2_train'
data_dir_dict = {}

# val
data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_val_dict_small_300')
data_dir_dict['music_dict'] = os.path.join(OPT_INDEX_1, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(OPT_INDEX_1, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(OPT_INDEX_1, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(OPT_INDEX_1, 'rir_dict')

In [124]:
data_len_dict = {}


data_len_dict['music_len'] = os.path.join(OPT_INDEX_1, 'music_len')
data_len_dict['noise_len'] = os.path.join(OPT_INDEX_1, 'noise_len')
data_len_dict['babble_len'] = os.path.join(OPT_INDEX_1, 'babble_len')

data_len_dict['spk2utt_train_len'] = os.path.join(OPT_INDEX_2, 'spk2utt_train_len')

In [125]:
config = {}

config['sr'] = 16000
config['repeats'] = 4
config['batch_size'] = 1
config['extended_prefectch'] = 1.0
config['out_dir'] = '/Lun0/zhiyong/sdsvc_t2_small_sparse/val_data_small_300/'

def iter_data_preload(dataset, i):
    dataset.get_random_list()
    start_time = time.time()
    for count, (data, label) in enumerate(dataset):
        with open(config['out_dir']+str(i)+'_'+str(count), 'wb') as handle:
            pickle.dump((data.astype(np.float16), label.astype(np.int16)), handle)
        print(time.time()-start_time)
        start_time = time.time()

dataset = VoxIterableDataset(data_dir_dict, data_len_dict, config)
dataset.noise_data_preload()

processes = [Process(target = iter_data_preload, args = (dataset, i)) for i in range(1)]
start_time = time.time()
[p.start() for p in processes]
joined = [p.join() for p in processes]
print(time.time()-start_time)

preloading music_dict
100
200
300
400
500
600
preloading noise_dict
100
200
300
400
500
600
700
800
900
preloading babble_dict
100
200
300
400
0.044844865798950195
0.01967334747314453
0.026225566864013672
0.013866186141967773
0.012897491455078125
0.033968448638916016
0.020188570022583008
0.012358903884887695
0.022062301635742188
0.013831853866577148
0.018858909606933594
0.03190279006958008
0.025790691375732422
0.010153055191040039
0.021881103515625
0.01739501953125
0.02304816246032715
0.023474454879760742
0.016068696975708008
0.01393437385559082
0.009956836700439453
0.013659954071044922
0.02497124671936035
0.019559860229492188
0.012761354446411133
0.0076923370361328125
0.021914005279541016
0.008727550506591797
0.022600173950195312
0.011641979217529297
0.02487492561340332
0.012909412384033203
0.008728981018066406
0.007687568664550781
0.00862574577331543
0.014181375503540039
0.017548799514770508
0.007707834243774414
0.014236927032470703
0.022825241088867188
0.008647918701171875
0.0248558

0.01627063751220703
0.018077611923217773
0.013524055480957031
0.020382404327392578
0.021958112716674805
0.019270658493041992
0.014860868453979492
0.01914381980895996
0.02398681640625
0.0180513858795166
0.019965171813964844
0.02671217918395996
0.013678550720214844
0.02978062629699707
0.01377105712890625
0.02782464027404785
0.022893190383911133
0.02252984046936035
0.018802881240844727
0.01373434066772461
0.015464305877685547
0.013599634170532227
0.04072427749633789
0.018343448638916016
0.029567480087280273
0.012468099594116211
0.012393951416015625
0.013583898544311523
0.015037775039672852
0.01502370834350586
0.012341022491455078
0.013663530349731445
0.03821587562561035
0.013711214065551758
0.020045042037963867
0.013619422912597656
0.026456832885742188
0.026402950286865234
0.016361713409423828
0.013556718826293945
0.01649022102355957
0.01719045639038086
0.0277862548828125
0.01362752914428711
0.014974117279052734
0.012334108352661133
0.01486063003540039
0.015373945236206055
0.0360867977142

0.01368570327758789
0.03209710121154785
0.01251220703125
0.029120206832885742
0.038652658462524414
0.015243053436279297
0.01354527473449707
0.033035993576049805
0.02943277359008789
0.013700246810913086
0.012354850769042969
0.014979362487792969
0.014199972152709961
0.013575077056884766
0.02901911735534668
0.01300954818725586
0.018959522247314453
0.012325286865234375
0.028142929077148438
0.01363992691040039
0.013623714447021484
0.023122787475585938
0.013648509979248047
0.01727914810180664
0.012339115142822266
0.01802682876586914
0.016214370727539062
0.012389659881591797
0.013550996780395508
0.013600349426269531
0.014154195785522461
0.01943230628967285
0.017089366912841797
0.025397062301635742
0.012424945831298828
0.01493978500366211
0.01947808265686035
0.013549566268920898
0.026096820831298828
0.013637304306030273
0.025259017944335938
0.012410402297973633
0.014222383499145508
0.013002395629882812
0.027624130249023438
0.01726245880126953
0.013594388961791992
0.03023672103881836
0.02840423

0.01346898078918457
0.028629541397094727
0.02615833282470703
0.02214217185974121
0.014951467514038086
0.025661468505859375
0.014370918273925781
0.015151023864746094
0.01363229751586914
0.01361083984375
0.024712562561035156
0.013620853424072266
0.012368917465209961
0.013590812683105469
0.027046680450439453
0.012314796447753906
24.38580894470215


# Make validation index (Need only once)

In [126]:
OPT_INDEX = '/Lun0/zhiyong/sdsvc_t2_small_sparse'
train_data_dir = os.path.join(OPT_INDEX, 'val_data_small_300')
expected_len = 300 * 4
workers = 1
single_worker_len = int(expected_len / workers)
output = os.path.join(OPT_INDEX, 'val_data_small_300.csv')

In [127]:
assert expected_len == len(glob.glob(train_data_dir+'/*'))

In [128]:
count = 0
with open(output, 'w') as f:
    for i in range(workers):
        for j in range(single_worker_len):
            path = os.path.join(train_data_dir, str(i)+'_'+str(j))
            assert os.path.isfile(path)
            f.write(path+'\n')
            count += 1

In [129]:
count

1200